In [2]:
import pandas as pd
import numpy as np
import re
import json
import ast
import tweepy
from textblob import TextBlob
import os
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.stem import SnowballStemmer
import emoji
import yake
#from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

As we are still unsure of the data we want, and are subject to changing our minds, I will be dropping irrelevant columns at the very end. This will allow us to quickly re-add columns in case we do change our minds.

In [3]:
#reading in activity watch data

activity_watch = pd.read_json("data/activity_watch_2021_04_18.json")


In [4]:
#separating the buckets column as that is where the dictionary of relevant data is

activity_watch = activity_watch['buckets'].apply(pd.Series)
activity_watch

,id,created,name,type,client,hostname,events
aw-stopwatch,aw-stopwatch,2021-02-27T07:11:34.038428+00:00,None,general.stopwatch,aw-webui,unknown,[]
aw-watcher-afk_DESKTOP-TOACMEE,aw-watcher-afk_DESKTOP-TOACMEE,2021-02-27T07:07:20.676248+00:00,None,afkstatus,aw-watcher-afk,DESKTOP-TOACMEE,[{'timestamp': '2021-04-18T09:41:13.362000+00:...
aw-watcher-window_DESKTOP-TOACMEE,aw-watcher-window_DESKTOP-TOACMEE,2021-02-27T07:07:20.499312+00:00,None,currentwindow,aw-watcher-window,DESKTOP-TOACMEE,[{'timestamp': '2021-04-18T14:15:21.485000+00:...


In [5]:
#Further splitting the events column to extract the time stamp, duration and data

activity_watch_split = pd.DataFrame(activity_watch['events'][2])
activity_watch_split

,timestamp,duration,data
0,2021-04-18T14:15:21.485000+00:00,10.812,"{'app': 'msedge.exe', 'title': 'ActivityWatch ..."
1,2021-04-18T14:15:20.283000+00:00,0.000,"{'app': 'aw-qt.exe', 'title': 'aw-qt'}"
2,2021-04-18T14:15:17.916000+00:00,1.196,"{'app': 'explorer.exe', 'title': ''}"
3,2021-04-18T14:15:16.716000+00:00,0.000,"{'app': 'msedge.exe', 'title': 'Oura API and 7..."
4,2021-04-18T14:15:15.525000+00:00,0.000,"{'app': 'msedge.exe', 'title': 'localhost:7777..."
...,...,...,...
26718,2021-02-27T15:09:10.151000+00:00,4.846,"{'app': 'SearchApp.exe', 'title': 'Search'}"
26719,2021-02-27T15:08:49.183000+00:00,19.718,"{'app': 'msedge.exe', 'title': 'Sponsor @Activ..."
26720,2021-02-27T15:08:24.630000+00:00,23.357,"{'app': 'msedge.exe', 'title': 'ActivityWatch ..."
26721,2021-02-27T15:08:23.428000+00:00,0.000,"{'app': 'msedge.exe', 'title': 'https://github..."


In [6]:
#dropping the data column in a copy of this

activity_watch_split_1 = activity_watch_split.drop(columns=["data"])
activity_watch_split_1

,timestamp,duration
0,2021-04-18T14:15:21.485000+00:00,10.812
1,2021-04-18T14:15:20.283000+00:00,0.000
2,2021-04-18T14:15:17.916000+00:00,1.196
3,2021-04-18T14:15:16.716000+00:00,0.000
4,2021-04-18T14:15:15.525000+00:00,0.000
...,...,...
26718,2021-02-27T15:09:10.151000+00:00,4.846
26719,2021-02-27T15:08:49.183000+00:00,19.718
26720,2021-02-27T15:08:24.630000+00:00,23.357
26721,2021-02-27T15:08:23.428000+00:00,0.000


In [7]:
#splitting the data column of activity_watch_split, to extract the app and title being used

activity_watch_split_2 = activity_watch_split['data'].apply(pd.Series)
activity_watch_split_2

,app,title
0,msedge.exe,ActivityWatch and 79 more pages - Personal - M...
1,aw-qt.exe,aw-qt
2,explorer.exe,
3,msedge.exe,Oura API and 78 more pages - Personal - Micros...
4,msedge.exe,localhost:7777/callback?code=AQAAFYrVKDTnt6XUV...
...,...,...
26718,SearchApp.exe,Search
26719,msedge.exe,Sponsor @ActivityWatch on GitHub Sponsors and ...
26720,msedge.exe,ActivityWatch and 22 more pages - Personal - M...
26721,msedge.exe,https://github.com/ActivityWatch and 22 more p...


In [8]:
#combining the two split activity watch dataframes so that it contains both the timestamp, app, title and duration

activity_watch_final = pd.concat([activity_watch_split_1, activity_watch_split_2], axis=1)
activity_watch_final

,timestamp,duration,app,title
0,2021-04-18T14:15:21.485000+00:00,10.812,msedge.exe,ActivityWatch and 79 more pages - Personal - M...
1,2021-04-18T14:15:20.283000+00:00,0.000,aw-qt.exe,aw-qt
2,2021-04-18T14:15:17.916000+00:00,1.196,explorer.exe,
3,2021-04-18T14:15:16.716000+00:00,0.000,msedge.exe,Oura API and 78 more pages - Personal - Micros...
4,2021-04-18T14:15:15.525000+00:00,0.000,msedge.exe,localhost:7777/callback?code=AQAAFYrVKDTnt6XUV...
...,...,...,...,...
26718,2021-02-27T15:09:10.151000+00:00,4.846,SearchApp.exe,Search
26719,2021-02-27T15:08:49.183000+00:00,19.718,msedge.exe,Sponsor @ActivityWatch on GitHub Sponsors and ...
26720,2021-02-27T15:08:24.630000+00:00,23.357,msedge.exe,ActivityWatch and 22 more pages - Personal - M...
26721,2021-02-27T15:08:23.428000+00:00,0.000,msedge.exe,https://github.com/ActivityWatch and 22 more p...


Here I will be checking for any duplicates in the timestamp column, and any anomalies in the duration column. I will also be going over any nulls in the app and title column.

In [9]:
#checking for nulls in the title column that may not have been labelled as null

activity_watch_nulls = activity_watch_final[activity_watch_final['title'] == ""]
activity_watch_nulls


,timestamp,duration,app,title
2,2021-04-18T14:15:17.916000+00:00,1.196,explorer.exe,
60,2021-04-18T14:04:11.987000+00:00,0.000,explorer.exe,
85,2021-04-18T14:00:06.266000+00:00,0.000,explorer.exe,
87,2021-04-18T14:00:00.310000+00:00,2.361,explorer.exe,
129,2021-04-18T13:47:31.655000+00:00,0.000,explorer.exe,
...,...,...,...,...
26648,2021-02-27T15:23:45.428000+00:00,0.100,explorer.exe,
26684,2021-02-27T15:15:53.395000+00:00,0.000,explorer.exe,
26688,2021-02-27T15:15:44.524000+00:00,0.027,explorer.exe,
26692,2021-02-27T15:15:38.558000+00:00,0.042,explorer.exe,


In [10]:
#checking if these null columns have any useful value in the app column

activity_watch_nulls["app"].value_counts()

explorer.exe                                                      1430
unknown                                                            479
msedge.exe                                                         110
msrdc.exe                                                           48
CredentialUIBroker.exe                                              14
LockApp.exe                                                         11
dwm.exe                                                              8
Zoom.exe                                                             5
Code - Insiders.exe                                                  3
OpenWith.exe                                                         2
ScreenClippingHost.exe                                               2
Spotify.exe                                                          2
CodeSetup-insider-0640cdeb8ba34f2f00cf4de73783632c4a692c52.tmp       2
python.exe                                                           2
rundll

The majority of it is unlabelled browsing on internet browser. We have two options:

Option 1) Drop all these empty columns
Option 2) Keep these empty columns

This will depend on how much information the original "app" column gives us, so I will be checking that first.

In [11]:
activity_watch_final["app"].value_counts()[0:15]

msedge.exe                  15235
explorer.exe                 2907
Code.exe                     1656
unknown                      1437
msrdc.exe                    1304
Code - Insiders.exe          1190
LockApp.exe                   573
notepad.exe                   372
msrdcw.exe                    269
ApplicationFrameHost.exe      258
WindowsTerminal.exe           240
ShellExperienceHost.exe       213
Zoom.exe                      181
CredentialUIBroker.exe        140
Spotify.exe                   140
Name: app, dtype: int64

Given that the majority of apps are msedge and explorer, and the null "app" values represent 9% and 16% of the original "app" column, it is best that we keep this data, as they are statistically significant.

Now we will be checking for duplicates in the timestamp column.

In [12]:
#extracting just the duplicates in the timestamp column

activity_watch_final_duplicates = activity_watch_final[activity_watch_final.duplicated(subset=["timestamp"]) == True]
activity_watch_final_duplicates.duplicated(subset=["timestamp"]).value_counts()



False    437
True      57
dtype: int64

In [13]:
#diving deeper into the duplicates to see if other variables such as "app" and "title" are the same

activity_watch_final_duplicates[30:51]

,timestamp,duration,app,title
19361,2021-03-14T14:20:57.804000+00:00,311.013,Code - Insiders.exe,● scratch.ipynb - braindump - Visual Studio Co...
19362,2021-03-14T14:20:57.804000+00:00,300.541,Code - Insiders.exe,● scratch.ipynb - braindump - Visual Studio Co...
19363,2021-03-14T14:20:57.804000+00:00,290.180,Code - Insiders.exe,● scratch.ipynb - braindump - Visual Studio Co...
19364,2021-03-14T14:20:57.804000+00:00,279.835,Code - Insiders.exe,● scratch.ipynb - braindump - Visual Studio Co...
19365,2021-03-14T14:20:57.804000+00:00,269.534,Code - Insiders.exe,● scratch.ipynb - braindump - Visual Studio Co...
19366,2021-03-14T14:20:57.804000+00:00,259.147,Code - Insiders.exe,● scratch.ipynb - braindump - Visual Studio Co...
19367,2021-03-14T14:20:57.804000+00:00,248.822,Code - Insiders.exe,● scratch.ipynb - braindump - Visual Studio Co...
19368,2021-03-14T14:20:57.804000+00:00,238.499,Code - Insiders.exe,● scratch.ipynb - braindump - Visual Studio Co...
19369,2021-03-14T14:20:57.804000+00:00,228.142,Code - Insiders.exe,● scratch.ipynb - braindump - Visual Studio Co...
19370,2021-03-14T14:20:57.804000+00:00,217.765,Code - Insiders.exe,● scratch.ipynb - braindump - Visual Studio Co...


In [14]:
#taking only the maximum value duplicate

activity_watch_final = activity_watch_final.sort_values('duration').drop_duplicates('timestamp', keep='last')
activity_watch_final

,timestamp,duration,app,title
9520,2021-04-05T13:36:32.387000+00:00,0.000,unknown,unknown
21612,2021-03-11T02:41:42.980000+00:00,0.000,explorer.exe,
11721,2021-03-31T14:39:21.656000+00:00,0.000,msedge.exe,bc hydro - transfer account - residential only...
4882,2021-04-15T05:26:09.626000+00:00,0.000,msedge.exe,New tab and 17 more pages - Personal - Microso...
11724,2021-03-31T14:38:54.668000+00:00,0.000,msedge.exe,WhatsApp Web and 11 more pages - Personal - Mi...
...,...,...,...,...
17935,2021-03-15T04:57:50.661000+00:00,36343.626,unknown,
13081,2021-03-27T02:29:07.006000+00:00,42341.804,LockApp.exe,Windows Default Lock Screen
13334,2021-03-25T23:30:00.056000+00:00,48138.768,LockApp.exe,Windows Default Lock Screen
11910,2021-03-30T23:44:12.699000+00:00,50594.155,LockApp.exe,Windows Default Lock Screen


In [15]:
#renaming duration and timestamp column to reflect the appropriate units

activity_watch_final.rename(columns={"duration":"duration_seconds"}, inplace=True)
activity_watch_final.rename(columns={"timestamp":"timestamp_utc"}, inplace=True)
activity_watch_final


,timestamp_utc,duration_seconds,app,title
9520,2021-04-05T13:36:32.387000+00:00,0.000,unknown,unknown
21612,2021-03-11T02:41:42.980000+00:00,0.000,explorer.exe,
11721,2021-03-31T14:39:21.656000+00:00,0.000,msedge.exe,bc hydro - transfer account - residential only...
4882,2021-04-15T05:26:09.626000+00:00,0.000,msedge.exe,New tab and 17 more pages - Personal - Microso...
11724,2021-03-31T14:38:54.668000+00:00,0.000,msedge.exe,WhatsApp Web and 11 more pages - Personal - Mi...
...,...,...,...,...
17935,2021-03-15T04:57:50.661000+00:00,36343.626,unknown,
13081,2021-03-27T02:29:07.006000+00:00,42341.804,LockApp.exe,Windows Default Lock Screen
13334,2021-03-25T23:30:00.056000+00:00,48138.768,LockApp.exe,Windows Default Lock Screen
11910,2021-03-30T23:44:12.699000+00:00,50594.155,LockApp.exe,Windows Default Lock Screen


In [16]:
#checking for numerical anomalies

activity_watch_final["duration_seconds"].describe()


count    26229.000000
mean        91.257934
std        975.146928
min          0.000000
25%          0.000000
50%          3.441000
75%         14.448000
max      53778.259000
Name: duration_seconds, dtype: float64

I think we should remove titles with a duration of 0 seconds.

In [17]:
#checking for numerical anomalies that look too small

activity_watch_final[activity_watch_final["duration_seconds"] == 0]


,timestamp_utc,duration_seconds,app,title
9520,2021-04-05T13:36:32.387000+00:00,0.0,unknown,unknown
21612,2021-03-11T02:41:42.980000+00:00,0.0,explorer.exe,
11721,2021-03-31T14:39:21.656000+00:00,0.0,msedge.exe,bc hydro - transfer account - residential only...
4882,2021-04-15T05:26:09.626000+00:00,0.0,msedge.exe,New tab and 17 more pages - Personal - Microso...
11724,2021-03-31T14:38:54.668000+00:00,0.0,msedge.exe,WhatsApp Web and 11 more pages - Personal - Mi...
...,...,...,...,...
19627,2021-03-13T21:36:04.150000+00:00,0.0,Code.exe,utils.py - eeg-notebooks - Visual Studio Code
15814,2021-03-20T20:56:14.954000+00:00,0.0,Code - Insiders.exe,gpt.py - braindump - Visual Studio Code - Insi...
2256,2021-04-17T19:34:22.748000+00:00,0.0,Code.exe,research.ts - neurosity-research-program-vscod...
19630,2021-03-13T21:35:00.531000+00:00,0.0,EXCEL.EXE,Find and Replace


In [18]:
#checking for numerical anomalies that look too big

activity_watch_final[activity_watch_final["duration_seconds"] > 14].sort_values("duration_seconds", ascending=False)[30:50]

,timestamp_utc,duration_seconds,app,title
8058,2021-04-10T00:35:38.425000+00:00,10773.160,unknown,unknown
23762,2021-03-03T00:52:05.165000+00:00,10194.149,unknown,
8738,2021-04-06T16:32:58.208000+00:00,9550.460,msrdc.exe,MININT-IICQLJG.northamerica.corp.microsoft.com...
13092,2021-03-26T17:57:45.689000+00:00,9540.321,msrdc.exe,MININT-IICQLJG.northamerica.corp.microsoft.com...
11355,2021-03-31T22:33:55.816000+00:00,9118.532,msrdc.exe,MININT-IICQLJG.northamerica.corp.microsoft.com...
7737,2021-04-11T03:43:24.130000+00:00,9112.638,unknown,unknown
17694,2021-03-16T18:54:50.814000+00:00,8965.096,msrdc.exe,MININT-IICQLJG.northamerica.corp.microsoft.com...
13090,2021-03-26T20:36:49.731000+00:00,8717.533,msrdc.exe,MININT-IICQLJG.northamerica.corp.microsoft.com...
14130,2021-03-23T17:36:16.382000+00:00,8474.894,msrdc.exe,MININT-IICQLJG.northamerica.corp.microsoft.com...
11329,2021-04-01T01:48:43.010000+00:00,8368.512,msrdc.exe,MININT-IICQLJG.northamerica.corp.microsoft.com...


Time spent on LockApp.exe is time spent on the laptops homepage, therefore we will be removing it.

In [19]:
#removing rows with LockApp

activity_watch_final.drop(index=activity_watch_final[activity_watch_final['app'] == 'LockApp.exe'].index, inplace=True)



In [20]:
#removing rows with unknown

activity_watch_final.drop(index=activity_watch_final[activity_watch_final['app'] == 'unknown'].index, inplace=True)

In [21]:
#checking for different variations of the same name
activity_watch_final["app"].value_counts()

msedge.exe                                                        14985
explorer.exe                                                       2858
Code.exe                                                           1629
msrdc.exe                                                          1303
Code - Insiders.exe                                                1093
notepad.exe                                                         363
msrdcw.exe                                                          269
ApplicationFrameHost.exe                                            256
WindowsTerminal.exe                                                 239
ShellExperienceHost.exe                                             212
Zoom.exe                                                            181
Spotify.exe                                                         140
CredentialUIBroker.exe                                              137
SearchApp.exe                                                   

In [22]:
#sorting the values from the earliest date to the latest date
activity_watch_final.sort_values("timestamp_utc", ascending=True, inplace=True)

In [23]:
#resetting the index values
activity_watch_final.reset_index(drop=True, inplace=True)
activity_watch_final

,timestamp_utc,duration_seconds,app,title
0,2021-02-27T15:07:29.345000+00:00,52.882,msedge.exe,ActivityWatch/activitywatch: The best free and...
1,2021-02-27T15:08:23.428000+00:00,0.000,msedge.exe,https://github.com/ActivityWatch and 22 more p...
2,2021-02-27T15:08:24.630000+00:00,23.357,msedge.exe,ActivityWatch and 22 more pages - Personal - M...
3,2021-02-27T15:08:49.183000+00:00,19.718,msedge.exe,Sponsor @ActivityWatch on GitHub Sponsors and ...
4,2021-02-27T15:09:10.151000+00:00,4.846,SearchApp.exe,Search
...,...,...,...,...
24240,2021-04-18T14:15:15.525000+00:00,0.000,msedge.exe,localhost:7777/callback?code=AQAAFYrVKDTnt6XUV...
24241,2021-04-18T14:15:16.716000+00:00,0.000,msedge.exe,Oura API and 78 more pages - Personal - Micros...
24242,2021-04-18T14:15:17.916000+00:00,1.196,explorer.exe,
24243,2021-04-18T14:15:20.283000+00:00,0.000,aw-qt.exe,aw-qt


In [24]:
#For names that include "code" I will be changing it to "VSCode" to avoid double counting

for index in range(len(activity_watch_final["app"])):
    if "Code" in activity_watch_final["app"][index]:
        activity_watch_final["app"][index] = "VSCode"


<ipython-input-24-4f330313c903>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  activity_watch_final["app"][index] = "VSCode"


In [25]:
#For names that include "spotify" I will be changing it to "Spotify" to avoid double counting

for index in range(len(activity_watch_final["app"])):
    if "Spotify" in activity_watch_final["app"][index] or "spotify" in activity_watch_final["app"][index]:
        activity_watch_final["app"][index] = "Spotify"


<ipython-input-25-b079fdea2fea>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  activity_watch_final["app"][index] = "Spotify"


In [26]:
#For titles that are empty I am converting them to null values

for index in range(len(activity_watch_final["title"])):
    if len(activity_watch_final["title"][index]) == 0:
        activity_watch_final["title"][index] = np.nan

<ipython-input-26-773baae9142f>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  activity_watch_final["title"][index] = np.nan


In [27]:
activity_watch_final[0:30]

,timestamp_utc,duration_seconds,app,title
0,2021-02-27T15:07:29.345000+00:00,52.882,msedge.exe,ActivityWatch/activitywatch: The best free and...
1,2021-02-27T15:08:23.428000+00:00,0.000,msedge.exe,https://github.com/ActivityWatch and 22 more p...
2,2021-02-27T15:08:24.630000+00:00,23.357,msedge.exe,ActivityWatch and 22 more pages - Personal - M...
3,2021-02-27T15:08:49.183000+00:00,19.718,msedge.exe,Sponsor @ActivityWatch on GitHub Sponsors and ...
4,2021-02-27T15:09:10.151000+00:00,4.846,SearchApp.exe,Search
5,2021-02-27T15:09:16.200000+00:00,13.051,explorer.exe,NaN
6,2021-02-27T15:09:30.308000+00:00,3.712,aw-qt.exe,aw-qt
7,2021-02-27T15:09:35.229000+00:00,0.104,msedge.exe,Untitled and 23 more pages - Personal - Micros...
8,2021-02-27T15:09:36.444000+00:00,0.089,msedge.exe,localhost:5600 and 23 more pages - Personal - ...
9,2021-02-27T15:09:37.642000+00:00,22.687,msedge.exe,ActivityWatch and 23 more pages - Personal - M...


In [28]:
#double checking for values that may be the same but have different names
activity_watch_final["timestamp_utc"].value_counts()

2021-04-06T14:35:33.114000+00:00    1
2021-04-11T16:06:02.719000+00:00    1
2021-03-15T01:58:31.812000+00:00    1
2021-02-27T18:25:32.635000+00:00    1
2021-03-19T04:27:32.024000+00:00    1
                                   ..
2021-04-12T13:47:35.201000+00:00    1
2021-04-12T13:56:26.039000+00:00    1
2021-04-18T12:22:28.746000+00:00    1
2021-04-14T07:16:45.884000+00:00    1
2021-04-08T21:36:25.020000+00:00    1
Name: timestamp_utc, Length: 24245, dtype: int64

In [29]:
activity_watch_final[activity_watch_final["app"]== "msedge.exe"].iloc[0:30]

,timestamp_utc,duration_seconds,app,title
0,2021-02-27T15:07:29.345000+00:00,52.882,msedge.exe,ActivityWatch/activitywatch: The best free and...
1,2021-02-27T15:08:23.428000+00:00,0.000,msedge.exe,https://github.com/ActivityWatch and 22 more p...
2,2021-02-27T15:08:24.630000+00:00,23.357,msedge.exe,ActivityWatch and 22 more pages - Personal - M...
3,2021-02-27T15:08:49.183000+00:00,19.718,msedge.exe,Sponsor @ActivityWatch on GitHub Sponsors and ...
7,2021-02-27T15:09:35.229000+00:00,0.104,msedge.exe,Untitled and 23 more pages - Personal - Micros...
8,2021-02-27T15:09:36.444000+00:00,0.089,msedge.exe,localhost:5600 and 23 more pages - Personal - ...
9,2021-02-27T15:09:37.642000+00:00,22.687,msedge.exe,ActivityWatch and 23 more pages - Personal - M...
10,2021-02-27T15:10:01.559000+00:00,0.031,msedge.exe,Features - ActivityWatch Forum and 23 more pag...
11,2021-02-27T15:10:02.769000+00:00,17.903,msedge.exe,Top Features topics - ActivityWatch Forum and ...
12,2021-02-27T15:10:21.865000+00:00,13.100,msedge.exe,ActivityWatch and 23 more pages - Personal - M...


In [30]:
#extracting key topics using yake

language = "en"
max_ngram_size = 3
deduplication_threshold = 0.9
numOfKeywords = 5 # the max number of results to expect

In [31]:
#creating a function that removes key words with low value, and then extracting key words for titles with "msedge" as the named app
browsing_yake_extractor = yake.KeywordExtractor(lan=language, n=max_ngram_size, dedupLim=deduplication_threshold, top=numOfKeywords, features=None)

activity_watch_edge_events = activity_watch_final[activity_watch_final["app"] == "msedge.exe"]

skip_words = ["Personal", "Microsoft", "Edge", "pages", "tab", "slack", "Ọrẹ̀", "Ore", "oreHGA", "April", "Ogundipe", "Timi", "ORE_OGUNDIPE_T4.pdf", "oreHGA", "(Ore Ogundipe)", "Ọrẹ̀ Ògúndípẹ̀", "API" ]

def getkeywords(text):
    #TODO: get rid of stop words before doing extraction
    # - punctuations
    # - "Personal - Microsoft Edge"
    if type(text) is not str:
        return None

    text = text.replace(" - Personal - Microsoft Edge", "")
    text = text.replace("slack", "")
    text = text.replace("Slack", "")
    text = text.replace("Twitter", "")
    text = text.replace("tweets", "")
    text = text.replace("Ore", "")
    text = text.replace("Ọrẹ̀", "")
    text = text.replace("Orehga", "")
    text = text.replace("oreHGA", "")
    text = text.replace("April", "")
    text = text.replace("Ogundipe", "")
    text = text.replace("Timi", "")
    text = text.replace("ORE_OGUNDIPE_T4.pdf", "")
    text = text.replace("(Ore Ogundipe)", "")
    text = text.replace("Ọrẹ̀ Ògúndípẹ̀", "")
    text = text.replace("buraksekili/", "")

    yake_keywords = browsing_yake_extractor.extract_keywords(text)

    resulting_keywords = []
    for result in yake_keywords:
        if result[0] in skip_words:
            continue
        # convert all keyword entries to lower case
        # each keyword has a structure of (keyword, confidence)
        resulting_keywords.append((result[0].lower(), result[1]))
    
    return resulting_keywords

activity_watch_edge_events["keywords"] = activity_watch_edge_events["title"].apply(lambda x: getkeywords(x))
activity_watch_edge_events["keywords"]

<ipython-input-31-7ce9b625d832>:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  activity_watch_edge_events["keywords"] = activity_watch_edge_events["title"].apply(lambda x: getkeywords(x))


0        [(automated time tracker, 0.008757238666059378...
1                                                       []
2                   [(activitywatch, 0.15831692877998726)]
3        [(activitywatch on github, 0.02111487214632003...
7                        [(untitled, 0.15831692877998726)]
                               ...                        
24233                        [(oura, 0.15831692877998726)]
24234                   [(oura api, 0.023458380875189744)]
24240    [(localhost, 0.04491197687864554), (callback, ...
24241                   [(oura api, 0.023458380875189744)]
24244               [(activitywatch, 0.15831692877998726)]
Name: keywords, Length: 14985, dtype: object

In [34]:
# we then go ahead to a frequency mapping
browser_keyword_freq_dict = {}
for keyword_entries in activity_watch_edge_events["keywords"]:
    if keyword_entries is None:
        continue
    
    # each keyword has a structure of (keyword, confidence)
    # we don't need confidence so we only take keyword
    for single_keyword_entry in keyword_entries:
        keyword = single_keyword_entry[0]
        if browser_keyword_freq_dict.get(keyword) == None:
            browser_keyword_freq_dict[keyword] = 1
        else:
            browser_keyword_freq_dict[keyword] += 1

# the result of browser_keyword_freq_dict has a strcuture of (keyword, appearance_count)
sorted_browser_keyword_freq_dict = dict(sorted(browser_keyword_freq_dict.items(), key=lambda item: item[1], reverse=True))

the result of browser_keyword_freq_dict has a strcuture of (keyword, appearance_count)

now we select a single winner called "context" from the keyword entries

In [35]:
#choosing the most relevant topic out of all the selected topics

def choose_winning_key(keyword_entries):
    """This is chosen as a function of occurence in total data set
    and confidence"""
    if keyword_entries is None or len(keyword_entries) < 1:
        return None
        
    keyword_weights_dict = {}
    # check frequency count for each word in descending order of confidence
    for entry in keyword_entries:
        # essentially weight = confidence * frequency
        keyword_weights_dict[entry[0]] = entry[1] * browser_keyword_freq_dict[entry[0]]

    # the dictionary will be of structure {"event_keyword", "weight"}
    # pick the keyword with the highest weight
    sorted_keyword_weights_dict = dict(sorted(keyword_weights_dict.items(), key=lambda item: item[1], reverse=True))
    
    #TODO:what happens when there's a tie - for now we're just picking the heighest
    return list(sorted_keyword_weights_dict.keys())[0]
    


activity_watch_edge_events["event_classification"] = activity_watch_edge_events["keywords"].apply(choose_winning_key)

activity_watch_edge_events["event_classification"].value_counts()

<ipython-input-35-f0855efa2127>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  activity_watch_edge_events["event_classification"] = activity_watch_edge_events["keywords"].apply(choose_winning_key)


meet               652
tweets             622
neurotechx         445
gmail              381
bing               380
                  ... 
wrong                1
seeingprogress       1
rapid automatic      1
join                 1
sachet jimjim        1
Name: event_classification, Length: 1476, dtype: int64

In [36]:
#seeing what the topics are for different rows of data

test_index = 6921
print(activity_watch_edge_events.iloc[test_index]["title"])
print(activity_watch_edge_events.iloc[test_index]["keywords"])
print(activity_watch_edge_events.iloc[test_index]["event_classification"])
# activity_watch_edge_events["event_classification"][420]

Short Talk: Fitt's Law & Text Input: Phrase Sets for Evaluating Text Entry Techniques and 10 more pages - Personal - Microsoft​ Edge
[('evaluating text entry', 0.0010431679112017647), ('text entry techniques', 0.0010431679112017647), ('fitt law', 0.0037296186725958302), ('phrase sets', 0.00557527170009609), ('short talk', 0.006586070833152184)]
phrase sets


In [37]:
activity_watch_edge_events[["title", "event_classification"]][9500:9550]

,title,event_classification
15205,Slack | openbci-challenge | NeuroTechX and 3 m...,neurotechx
15206,Slack | Cris | NeuroTechX and 3 more pages - P...,neurotechx
15207,New tab and 4 more pages - Personal - Microsof...,None
15208,https://web.whatsapp.com and 4 more pages - Pe...,None
15209,WhatsApp and 4 more pages - Personal - Microso...,whatsapp
15211,Slack | Cris | NeuroTechX and 3 more pages - P...,neurotechx
15212,(12) WhatsApp - Personal - Microsoft​ Edge,whatsapp
15213,Untitled and 1 more page - Personal - Microsof...,untitled
15214,Launch Meeting - Zoom and 1 more page - Person...,meeting
15216,Launch Meeting - Zoom and 1 more page - Person...,meeting


Now doing the same for VSCode

In [38]:
#Removing words from VSCode that gives us little information, and classifying based on hihg value words

activity_watch_events_vscode = activity_watch_final[activity_watch_final["app"] == "VSCode"]

def get_vscode_context(text):
    if type(text) is not str:
        return (None, None)
    # output from this will be "filename", "project"
    # first split string with space " - "
    text = text.replace("Visual Studio Code - Insiders", "")
    text = text.replace("Visual Studio Code", "")
    text = text.replace("(Untracked)", "")
    text = text.replace("(Working Tree)", "")
    text = text.replace("●", "")
    text = text.rstrip(" - ")

    text_split = text.split(" - ")
    # sort in descending order
    reverse_text_split_sort = sorted(text_split, key=text_split.index, reverse=True)

    # item 0 will be project name
    # item 1 will be the file name
    project = None
    filename = None

    if len(reverse_text_split_sort) == 1:
        project = reverse_text_split_sort[0]

    if len(reverse_text_split_sort) >= 2:
        project = reverse_text_split_sort[0]
        filename = reverse_text_split_sort[1]

    # return a tuple "(project, filename)"
    return (project, filename)


# calling the column keywords to be consistent with activity watch
activity_watch_events_vscode["keywords"] = activity_watch_events_vscode["title"].apply(get_vscode_context)

<ipython-input-38-ee22ffc3d24a>:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  activity_watch_events_vscode["keywords"] = activity_watch_events_vscode["title"].apply(get_vscode_context)


for now, we will only use the project as the event classification

In [39]:
#seeing what the data looks like
activity_watch_events_vscode["event_classification"] = activity_watch_events_vscode["keywords"].apply(lambda x: x[0])

activity_watch_events_vscode["event_classification"].value_counts()

<ipython-input-39-551cddc270c7>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  activity_watch_events_vscode["event_classification"] = activity_watch_events_vscode["keywords"].apply(lambda x: x[0])


braindump                               1034
fusion                                   965
eeg-notebooks                            230
akb-intel                                125
                                          94
neurosity-research-program-vscode         72
Welcome                                   33
webvr-musicalforest                       31
neoish                                    29
gpt3-sandbox                              27
Setup                                     25
Getting Started                           21
Open Folder                               12
blog                                       9
testing4.xdf                               7
Save As                                    5
Release Notes: 1.54.0-insider              4
Extension: Live Share Extension Pack       3
Release Notes: 1.54.1                      3
Open File                                  3
Select Python interpreter                  2
Release Notes: 1.53.2                      2
Open Works

In [40]:
#combine vscode & msedge events

combined_aw_edge_vscode =  activity_watch_edge_events.append(activity_watch_events_vscode)
combined_aw_edge_vscode_final = combined_aw_edge_vscode.drop(["timestamp_utc", "app", "duration_seconds", "title"], axis=1)
combined_aw_edge_vscode_final

,keywords,event_classification
0,"[(automated time tracker, 0.008757238666059378...",activitywatch
1,[],None
2,"[(activitywatch, 0.15831692877998726)]",activitywatch
3,"[(activitywatch on github, 0.02111487214632003...",github sponsors
7,"[(untitled, 0.15831692877998726)]",untitled
...,...,...
24235,"(fusion, oura_2021-04-18T14-13-10.json)",fusion
24236,"(Save As, None)",Save As
24237,"(fusion, oura_2021-04-18T14-13-10.json)",fusion
24238,"(fusion, oura_data_ore_2021_04_18.json)",fusion


In [41]:

activity_watch_final[0:10]

,timestamp_utc,duration_seconds,app,title
0,2021-02-27T15:07:29.345000+00:00,52.882,msedge.exe,ActivityWatch/activitywatch: The best free and...
1,2021-02-27T15:08:23.428000+00:00,0.000,msedge.exe,https://github.com/ActivityWatch and 22 more p...
2,2021-02-27T15:08:24.630000+00:00,23.357,msedge.exe,ActivityWatch and 22 more pages - Personal - M...
3,2021-02-27T15:08:49.183000+00:00,19.718,msedge.exe,Sponsor @ActivityWatch on GitHub Sponsors and ...
4,2021-02-27T15:09:10.151000+00:00,4.846,SearchApp.exe,Search
5,2021-02-27T15:09:16.200000+00:00,13.051,explorer.exe,NaN
6,2021-02-27T15:09:30.308000+00:00,3.712,aw-qt.exe,aw-qt
7,2021-02-27T15:09:35.229000+00:00,0.104,msedge.exe,Untitled and 23 more pages - Personal - Micros...
8,2021-02-27T15:09:36.444000+00:00,0.089,msedge.exe,localhost:5600 and 23 more pages - Personal - ...
9,2021-02-27T15:09:37.642000+00:00,22.687,msedge.exe,ActivityWatch and 23 more pages - Personal - M...


In [42]:
#activity watch with final data

activity_watch_dataframe_final = pd.concat([activity_watch_final, combined_aw_edge_vscode_final], axis=1, join="outer")
activity_watch_dataframe_final["event_classification"].value_counts()[30:60]

notion                               89
github                               88
status                               86
vauban                               78
jmperez                              73
neurosity-research-program-vscode    72
google developers                    70
collection                           68
seaborn                              67
sharing                              67
openai-api                           62
google                               62
issue                                62
fourier transform                    57
cloud platform                       55
dashboard                            54
knabu                                48
latest tweets                        47
general                              45
wikipedia                            45
microsoft                            45
inprivate                            44
tax                                  44
search                               41
canada revenue                       39


In [43]:
activity_watch_dataframe_final["event_classification"].value_counts()[0:30]

braindump                   1034
fusion                      1012
meet                         652
tweets                       622
neurotechx                   445
gmail                        381
bing                         380
google calendar              253
whatsapp                     243
untitled                     237
eeg-notebooks                230
notifications                192
ndi                          190
openai api                   187
neurosity                    184
documentation                167
questrade                    158
page                         141
linkedin                     141
adaobi                       133
akb-intel                    125
spotify for developers       115
mne                          113
eeg-notebooks developers     106
activitywatch                 97
stack overflow                97
future africa                 94
calendly                      94
                              94
google drive                  91
Name: even

In [44]:
activity_watch_dataframe_final["event_classification"].value_counts()[90:120]

account                       25
brain-dump                    25
afriknowledgebase             24
mne-tools                     24
redirecting                   23
computational neuroscience    23
davie flowers                 23
decide uploads                23
open source                   23
localhost                     22
api documentation             22
selection                     21
quantified self personal      21
pandas dataframe              21
pull request                  21
spotify streaming             21
spotify api                   21
Getting Started               21
work                          21
save                          20
crypto                        20
signal processing             20
paypal                        20
kumodzi                       19
alexcastillo                  19
deploying a flask             19
googlecreativelab             19
hydro                         19
https                         18
scipy                         18
Name: even

In [45]:
#replace mislablled titles with correct values

activity_watch_dataframe_final["title"].iloc[[5393, 5395, 5413]] = "neurotechx"
activity_watch_dataframe_final["title"].iloc[[12293,12295]] = "linkedin"
activity_watch_dataframe_final["title"].iloc[[3925,3930,3934,3938, 3940, 3944, 3947 ]] = "linkedin"


c:\python38\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [46]:
#changing mislaballed topics

def format_label(label):
    if label == "eeg-notebooks developers":
        label = "eeg-notebooks"
    if label == "calendly":
        label = "planning"
    if label == "google calendar":
        label = "planning"
    if label == "latest tweets":
        label = "tweets"
    if label == "brain-dump":
        label = "braindump"
    if label == "eeg-notebooks project":
        label = "eeg-notebooks"
    if label == "spotify streaming":
        label = "spotify"
    if label == "openai-api":
        label = "openai api"
        
        
    
        
    return label

activity_watch_dataframe_final["event_classification"] = activity_watch_dataframe_final["event_classification"].apply(format_label)

In [47]:
activity_watch_dataframe_final["event_classification"].value_counts()[0:60]

braindump                            1059
fusion                               1012
tweets                                669
meet                                  652
neurotechx                            445
gmail                                 381
bing                                  380
planning                              347
eeg-notebooks                         346
openai api                            249
whatsapp                              243
untitled                              237
notifications                         192
ndi                                   190
neurosity                             184
documentation                         167
questrade                             158
linkedin                              141
page                                  141
adaobi                                133
akb-intel                             125
spotify for developers                115
mne                                   113
stack overflow                    

In [48]:
#making sure everything is in lowercase
activity_watch_dataframe_final["event_classification"] = activity_watch_dataframe_final["event_classification"].str.lower()
activity_watch_dataframe_final["event_classification"].value_counts()

braindump            1059
fusion               1012
tweets                669
meet                  652
neurotechx            445
                     ... 
air street              1
orta                    1
listening history       1
hue-gah                 1
integration             1
Name: event_classification, Length: 1493, dtype: int64

In [49]:
#Filling in null values in the classifcation column, with values from the app column
activity_watch_dataframe_final["event_classification"].fillna(activity_watch_dataframe_final["app"], inplace=True)
activity_watch_dataframe_final["event_classification"].fillna(activity_watch_dataframe_final["app"], inplace=True)

In [50]:
def null_label(label):
    if label == "":
        label = "untitled"
    return label
        
activity_watch_dataframe_final["event_classification"] = activity_watch_dataframe_final["event_classification"].apply(null_label)


In [51]:
activity_watch_dataframe_final["event_classification"].value_counts()

explorer.exe     2858
msedge.exe       2121
msrdc.exe        1303
braindump        1059
fusion           1012
                 ... 
obiageli            1
accessibility       1
goals               1
simple              1
blocked             1
Name: event_classification, Length: 1534, dtype: int64

In [52]:
#reading google calendar data in

g_calendar = pd.read_csv("data/google_calendar_ore_2021_04_18.csv")
g_calendar.columns

Index(['kind', 'etag', 'id', 'status', 'htmlLink', 'created', 'updated',
       'summary', 'creator', 'organizer', 'start', 'end', 'recurringEventId',
       'originalStartTime', 'visibility', 'iCalUID', 'sequence', 'reminders',
       'eventType', 'description', 'colorId', 'transparency', 'location',
       'attendees', 'guestsCanInviteOthers', 'privateCopy', 'source',
       'hangoutLink', 'conferenceData', 'extendedProperties',
       'guestsCanModify', 'guestsCanSeeOtherGuests'],
      dtype='object')

In [53]:
#inspecting the data to identify any potential anomalies
g_calendar["created"].unique()

array(['2015-04-04T02:15:49.000Z', '2016-04-01T01:56:31.000Z',
       '2020-10-19T15:35:00.000Z', '2020-10-19T15:34:38.000Z',
       '2020-10-19T15:34:39.000Z', '2020-12-17T16:17:18.000Z',
       '2020-12-13T19:54:36.000Z', '2020-12-25T20:07:28.000Z',
       '2020-12-26T14:36:21.000Z', '2020-12-26T20:06:11.000Z',
       '2020-12-23T23:29:59.000Z', '2020-12-31T20:26:42.000Z',
       '2021-01-01T01:12:06.000Z', '2021-01-01T10:22:12.000Z',
       '2020-12-21T01:50:28.000Z', '2021-01-04T22:56:34.000Z',
       '2021-01-06T15:56:23.000Z', '2021-01-10T20:33:09.000Z',
       '2021-01-12T10:00:46.000Z', '2021-01-14T17:55:48.000Z',
       '2021-01-15T06:34:48.000Z', '2020-12-21T18:30:05.000Z',
       '2021-01-13T15:53:47.000Z', '2021-01-15T18:00:44.000Z',
       '2021-01-12T08:52:04.000Z', '2021-01-17T23:38:48.000Z',
       '2021-01-18T15:14:06.000Z', '2020-12-23T20:39:07.000Z',
       '2021-01-22T06:10:43.000Z', '2021-01-22T09:05:15.000Z',
       '2021-01-21T10:09:12.000Z', '2021-01-25T02:27:55

In [54]:
#inspecting the data to identify any potential anomalies
g_calendar["location"].value_counts()

Google Meet (instructions in description)                                                 59
Amsterdam AMS                                                                              1
https://us02web.zoom.us/j/84101445821                                                      1
https://meet.google.com/vra-emhm-zoo                                                       1
https://meet.google.com/kgk-qgha-gai                                                       1
Accra ACC                                                                                  1
https://zoom.us/j/97477945016?pwd=c1FLMWJjZDVucnFGY2k1MXV4MHpDUT09                         1
Schiphol Airport (Evert v/d Beekstraat 202, 1118 Amsterdam North Holland, Netherlands)     1
https://zoom.us/j/92547005201                                                              1
Kotoka International Airport (Accra Accra, Ghana)                                          1
Tribeca Hotel, Accra                                                  

In [55]:
#inspecting the data to identify any potential anomalies
type(g_calendar["start"][0])

str

The columns start and end both contain dictionaries, so I will be separating them into dataframes then recombining.

In [56]:
#As discovered above, the dictionaries have been stored as strings. I will need to convert them into dictionaries so that I can split the columns.
g_calendar_start = g_calendar["start"].apply(ast.literal_eval)
g_calendar_start

0      {'dateTime': '2021-04-14T16:00:00-07:00', 'tim...
1                                 {'date': '2021-03-15'}
2              {'dateTime': '2021-01-11T04:00:00-08:00'}
3      {'dateTime': '2021-01-10T14:15:00-08:00', 'tim...
4      {'dateTime': '2021-01-11T04:00:00-08:00', 'tim...
                             ...                        
252            {'dateTime': '2021-04-17T12:15:00-07:00'}
253            {'dateTime': '2021-04-17T18:15:00-07:00'}
254            {'dateTime': '2021-04-17T20:30:00-07:00'}
255            {'dateTime': '2021-04-17T21:00:00-07:00'}
256            {'dateTime': '2021-04-17T21:00:00-07:00'}
Name: start, Length: 257, dtype: object

In [57]:
#The start times for google calendar events
g_calendar_start = g_calendar_start.apply(pd.Series)
g_calendar_start.rename(columns={"dateTime": "start_dateTime_utc"}, inplace=True)
g_calendar_start

,start_dateTime_utc,timeZone,date
0,2021-04-14T16:00:00-07:00,Europe/Dublin,NaN
1,NaN,NaN,2021-03-15
2,2021-01-11T04:00:00-08:00,NaN,NaN
3,2021-01-10T14:15:00-08:00,Atlantic/Reykjavik,NaN
4,2021-01-11T04:00:00-08:00,Europe/Berlin,NaN
...,...,...,...
252,2021-04-17T12:15:00-07:00,NaN,NaN
253,2021-04-17T18:15:00-07:00,NaN,NaN
254,2021-04-17T20:30:00-07:00,NaN,NaN
255,2021-04-17T21:00:00-07:00,NaN,NaN


In [58]:
#splitting google calendar end date
g_calendar_end = g_calendar["end"].apply(ast.literal_eval)
g_calendar_end = g_calendar_end.apply(pd.Series)
g_calendar_end.rename(columns={"dateTime": "end_dateTime_utc"}, inplace=True)
g_calendar_end

,end_dateTime_utc,timeZone,date
0,2021-04-14T17:00:00-07:00,Europe/Dublin,NaN
1,NaN,NaN,2021-03-16
2,2021-01-11T13:55:00-08:00,NaN,NaN
3,2021-01-10T20:50:00-08:00,Europe/Berlin,NaN
4,2021-01-11T13:55:00-08:00,America/Vancouver,NaN
...,...,...,...
252,2021-04-17T18:00:00-07:00,NaN,NaN
253,2021-04-17T20:15:00-07:00,NaN,NaN
254,2021-04-17T21:00:00-07:00,NaN,NaN
255,2021-04-17T22:15:00-07:00,NaN,NaN


from looking at these different datetimes, it looks as though it is in UTC minus X hours, depending on the timezone it is in.

In [59]:
#combining both the start and end columns into one dataframe

g_calendar_times_combined = g_calendar_start.join(g_calendar_end, lsuffix='_act', rsuffix='_res')
g_calendar_times_combined

,start_dateTime_utc,timeZone_act,date_act,end_dateTime_utc,timeZone_res,date_res
0,2021-04-14T16:00:00-07:00,Europe/Dublin,NaN,2021-04-14T17:00:00-07:00,Europe/Dublin,NaN
1,NaN,NaN,2021-03-15,NaN,NaN,2021-03-16
2,2021-01-11T04:00:00-08:00,NaN,NaN,2021-01-11T13:55:00-08:00,NaN,NaN
3,2021-01-10T14:15:00-08:00,Atlantic/Reykjavik,NaN,2021-01-10T20:50:00-08:00,Europe/Berlin,NaN
4,2021-01-11T04:00:00-08:00,Europe/Berlin,NaN,2021-01-11T13:55:00-08:00,America/Vancouver,NaN
...,...,...,...,...,...,...
252,2021-04-17T12:15:00-07:00,NaN,NaN,2021-04-17T18:00:00-07:00,NaN,NaN
253,2021-04-17T18:15:00-07:00,NaN,NaN,2021-04-17T20:15:00-07:00,NaN,NaN
254,2021-04-17T20:30:00-07:00,NaN,NaN,2021-04-17T21:00:00-07:00,NaN,NaN
255,2021-04-17T21:00:00-07:00,NaN,NaN,2021-04-17T22:15:00-07:00,NaN,NaN


In [60]:
#combining the calendar times dataframe with the original dataframe

g_calendar_final = g_calendar.join(g_calendar_times_combined, lsuffix='_act', rsuffix='_res')
g_calendar_final

,kind,etag,id,status,htmlLink,created,updated,summary,creator,organizer,...,conferenceData,extendedProperties,guestsCanModify,guestsCanSeeOtherGuests,start_dateTime_utc,timeZone_act,date_act,end_dateTime_utc,timeZone_res,date_res
0,calendar#event,"""2885397680780000""",7ui7kk61env04pff2r4oet7ba4_20210414T230000Z,confirmed,https://www.google.com/calendar/event?eid=N3Vp...,2015-04-04T02:15:49.000Z,2016-03-06T17:05:31.131Z,Jemimah's Birthday,"{'email': 'oreogundipe@gmail.com', 'displayNam...","{'email': 'oreogundipe@gmail.com', 'displayNam...",...,NaN,NaN,NaN,NaN,2021-04-14T16:00:00-07:00,Europe/Dublin,NaN,2021-04-14T17:00:00-07:00,Europe/Dublin,NaN
1,calendar#event,"""2918951584034000""",ccsj6c9oc9j62b9j70qjeb9k6gp3ab9pchi3cb9m6so36e...,confirmed,https://www.google.com/calendar/event?eid=Y2Nz...,2016-04-01T01:56:31.000Z,2016-04-01T01:56:32.052Z,It's your day Capt. OG,"{'email': 'oreogundipe@gmail.com', 'displayNam...","{'email': 'oreogundipe@gmail.com', 'displayNam...",...,NaN,NaN,NaN,NaN,NaN,NaN,2021-03-15,NaN,NaN,2021-03-16
2,calendar#event,"""3211334650498000""",_6tlnaqrle5p6cpb4dhmj4phpehlmio9j6hlj4ord71jn0...,confirmed,https://www.google.com/calendar/event?eid=XzZ0...,2020-10-19T15:35:00.000Z,2020-12-17T13:49:10.095Z,Flight to Vancouver (KL 681),"{'email': 'oreogundipe@gmail.com', 'self': True}",{'email': 'unknownorganizer@calendar.google.com'},...,NaN,NaN,NaN,NaN,2021-01-11T04:00:00-08:00,NaN,NaN,2021-01-11T13:55:00-08:00,NaN,NaN
3,calendar#event,"""3211334719809000""",_60q30c1g60o30e1i60o4ac1g60rj8gpl88rj2c1h84s34...,confirmed,https://www.google.com/calendar/event?eid=XzYw...,2020-10-19T15:34:38.000Z,2020-12-17T13:50:40.936Z,KLM flight 590 to Amsterdam (N8JTST),"{'email': 'oreogundipe@gmail.com', 'self': True}","{'email': 'oreogundipe@gmail.com', 'self': True}",...,NaN,NaN,NaN,NaN,2021-01-10T14:15:00-08:00,Atlantic/Reykjavik,NaN,2021-01-10T20:50:00-08:00,Europe/Berlin,NaN
4,calendar#event,"""3211334720967000""",_60q30c1g60o30e1i60o4ac1g60rj8gpl88rj2c1h84s34...,confirmed,https://www.google.com/calendar/event?eid=XzYw...,2020-10-19T15:34:39.000Z,2020-12-17T13:50:41.515Z,KLM flight 681 to Vancouver (N8JTST),"{'email': 'oreogundipe@gmail.com', 'self': True}","{'email': 'oreogundipe@gmail.com', 'self': True}",...,NaN,NaN,NaN,NaN,2021-01-11T04:00:00-08:00,Europe/Berlin,NaN,2021-01-11T13:55:00-08:00,America/Vancouver,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
252,calendar#event,"""3237363132626000""",5mtl0s7s1l9f9d11584bmvvpih,confirmed,https://www.google.com/calendar/event?eid=NW10...,2021-04-17T17:46:06.000Z,2021-04-17T17:46:06.313Z,fusion - putting data together,"{'email': 'oreogundipe@gmail.com', 'self': True}","{'email': 'oreogundipe@gmail.com', 'self': True}",...,NaN,NaN,NaN,NaN,2021-04-17T12:15:00-07:00,NaN,NaN,2021-04-17T18:00:00-07:00,NaN,NaN
253,calendar#event,"""3237363179034000""",1961jk9r264751n3i683cdoi7g,confirmed,https://www.google.com/calendar/event?eid=MTk2...,2021-04-17T17:46:29.000Z,2021-04-17T17:46:29.517Z,go outside,"{'email': 'oreogundipe@gmail.com', 'self': True}","{'email': 'oreogundipe@gmail.com', 'self': True}",...,NaN,NaN,NaN,NaN,2021-04-17T18:15:00-07:00,NaN,NaN,2021-04-17T20:15:00-07:00,NaN,NaN
254,calendar#event,"""3237363211286000""",6o9tc9gtav6i52ond6tph8ujqq,confirmed,https://www.google.com/calendar/event?eid=Nm85...,2021-04-17T17:46:45.000Z,2021-04-17T17:46:45.643Z,somma - java homework,"{'email': 'oreogundipe@gmail.com', 'self': True}","{'email': 'oreogundipe@gmail.com', 'self': True}",...,NaN,NaN,NaN,NaN,2021-04-17T20:30:00-07:00,NaN,NaN,2021-04-17T21:00:00-07:00,NaN,NaN
255,calendar#event,"""3237363241070000""",6upb8fv4kkur6ugmunc8jsesm1,confirmed,https://www.google.com/calendar/event?eid=NnVw...,2021-04-17T17:47:00.000Z,2021-04-17T17:47:00.535Z,eat & nap,"{'email': 'oreogundipe@gmail.com', 'self': True}","{'email': 'oreogundipe@gmail.com', 'self': True}",...,NaN,NaN,NaN,NaN,2021-04-17T21:00:00-07:00,NaN,NaN,2021-04-17T22:15:00-07:00,NaN,NaN


In [61]:
g_calendar_final["timeZone_act"].value_counts()

America/Vancouver     112
Africa/Algiers         14
America/New_York        5
Europe/London           2
Europe/Berlin           1
Europe/Dublin           1
Atlantic/Reykjavik      1
Name: timeZone_act, dtype: int64

In [62]:
#here I will be dropping the irrelevant columns

Based on the results above I will only be keeping the following columns as they provide us with the most information:

- meeting start time (g_calendar_start)
- meeting end time (g_clanedar_end)
- timezone_act
- created
- summary
- description

In [64]:
#reading oura file in

oura_file = open('data/oura_data_ore_2021_04_18.json')
oura_json = oura_file.read()
oura_data = json.loads(oura_json)


In [65]:
#splitting oura_data into first dataframe which just has activity data. I will be combining it with other oura dataframes on the summary_date column as that column is in all dataframes

oura_activity = pd.DataFrame.from_records(oura_data["activity"])
oura_activity
oura_activity["summary_date"].describe()

count             84
unique            84
top       2021-04-07
freq               1
Name: summary_date, dtype: object

In [66]:
#extracting restufl periods into it's own dataframe

oura_restful_periods = pd.DataFrame.from_records(oura_data["restful_periods"])
oura_restful_periods
oura_restful_periods["summary_date"].describe()

count             52
unique            39
top       2021-02-07
freq               3
Name: summary_date, dtype: object

In [67]:
#extracting readiness into it's own dataframe

oura_readiness = pd.DataFrame.from_records(oura_data["readiness"])
type(oura_readiness["summary_date"][5])
oura_readiness

,period_id,rest_mode_state,score,score_activity_balance,score_hrv_balance,score_previous_day,score_previous_night,score_recovery_index,score_resting_hr,score_sleep_balance,score_temperature,summary_date
0,0,0,87,0,0,0,71,100,95,0,99,2021-01-21
1,1,0,73,0,0,90,62,48,90,0,100,2021-01-22
2,1,0,87,0,0,89,82,100,93,80,90,2021-01-23
3,2,0,81,92,0,93,70,81,95,70,98,2021-01-24
4,0,0,51,59,0,74,32,7,82,50,97,2021-01-25
...,...,...,...,...,...,...,...,...,...,...,...,...
77,1,0,47,36,70,87,39,57,50,19,92,2021-04-13
78,0,0,44,31,64,51,64,64,27,25,88,2021-04-14
79,1,0,54,40,66,80,46,74,76,25,100,2021-04-15
80,0,0,46,52,69,68,26,41,68,21,90,2021-04-16


In [68]:
#extracting sleep into it's own dataframe

oura_sleep = pd.DataFrame.from_records(oura_data["sleep"])
oura_sleep["summary_date"][3]

'2021-01-24'

In [69]:
#left joining datarame on the summary column first
oura_combined = oura_activity.join(oura_restful_periods, lsuffix='_act', rsuffix='_res')
oura_combined

,average_met,cal_active,cal_total,class_5min,daily_movement,day_end,day_start,high,inactive,inactivity_alerts,...,bedtime_end,bedtime_start,breath_average,duration,period_id,summary_date_res,timezone_res,hr_average,hr_lowest,rmssd
0,0.93750,23,1942,0000000000000000000000000000000000000000000000...,461,2021-01-21T03:59:59-08:00,2021-01-20T04:00:00-08:00,0,141,1,...,2021-01-21T07:36:24-08:00,2021-01-21T07:23:24-08:00,15.250,780.0,1.0,2021-01-21,-480.0,NaN,NaN,NaN
1,1.34375,234,2311,2232332222222221222222322222333332322223311221...,4151,2021-01-22T03:59:59-08:00,2021-01-21T04:00:00-08:00,1,792,1,...,2021-01-23T13:25:29-08:00,2021-01-23T13:09:29-08:00,15.125,960.0,3.0,2021-01-23,-480.0,59.00,58.0,64.0
2,1.34375,290,2350,1211222221111111122212211222222222222224434333...,5337,2021-01-23T03:59:59-08:00,2021-01-22T04:00:00-08:00,0,612,0,...,2021-01-23T14:50:29-08:00,2021-01-23T14:39:29-08:00,15.125,660.0,4.0,2021-01-23,-480.0,60.00,60.0,NaN
3,1.15625,96,2048,2232222211122211111111111111111111112211111111...,1832,2021-01-24T03:59:59-08:00,2021-01-23T04:00:00-08:00,0,517,1,...,2021-01-28T15:34:12-08:00,2021-01-28T15:06:12-08:00,15.250,1680.0,1.0,2021-01-28,-480.0,85.67,83.0,29.0
4,1.68750,707,2894,1111111111111111111111221212000000000221111112...,13257,2021-01-25T03:59:59-08:00,2021-01-24T04:00:00-08:00,2,703,0,...,2021-01-30T13:01:59-08:00,2021-01-30T12:40:59-08:00,14.875,1260.0,1.0,2021-01-30,-480.0,65.75,64.0,63.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,1.65625,721,2859,1111112112111112222222222222222323222222212211...,12814,2021-04-14T03:59:59-07:00,2021-04-13T04:00:00-07:00,20,710,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
80,1.40625,301,2400,1111111111111111111112111111111111111121121121...,5541,2021-04-15T03:59:59-07:00,2021-04-14T04:00:00-07:00,0,774,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
81,1.37500,193,2324,1111111111111112111112221222222222233333222222...,3148,2021-04-16T03:59:59-07:00,2021-04-15T04:00:00-07:00,1,1062,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
82,1.50000,550,2609,1111111111111111122111111111111111111221122211...,10674,2021-04-17T03:59:59-07:00,2021-04-16T04:00:00-07:00,2,451,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [70]:
#combining the readiness dataframe to the above combined dataframe
oura_combined_1 = oura_combined.join(oura_readiness, lsuffix='_act_1', rsuffix='_res_1')
oura_combined_1

,average_met,cal_active,cal_total,class_5min,daily_movement,day_end,day_start,high,inactive,inactivity_alerts,...,score_res_1,score_activity_balance,score_hrv_balance,score_previous_day,score_previous_night,score_recovery_index,score_resting_hr,score_sleep_balance,score_temperature,summary_date
0,0.93750,23,1942,0000000000000000000000000000000000000000000000...,461,2021-01-21T03:59:59-08:00,2021-01-20T04:00:00-08:00,0,141,1,...,87.0,0.0,0.0,0.0,71.0,100.0,95.0,0.0,99.0,2021-01-21
1,1.34375,234,2311,2232332222222221222222322222333332322223311221...,4151,2021-01-22T03:59:59-08:00,2021-01-21T04:00:00-08:00,1,792,1,...,73.0,0.0,0.0,90.0,62.0,48.0,90.0,0.0,100.0,2021-01-22
2,1.34375,290,2350,1211222221111111122212211222222222222224434333...,5337,2021-01-23T03:59:59-08:00,2021-01-22T04:00:00-08:00,0,612,0,...,87.0,0.0,0.0,89.0,82.0,100.0,93.0,80.0,90.0,2021-01-23
3,1.15625,96,2048,2232222211122211111111111111111111112211111111...,1832,2021-01-24T03:59:59-08:00,2021-01-23T04:00:00-08:00,0,517,1,...,81.0,92.0,0.0,93.0,70.0,81.0,95.0,70.0,98.0,2021-01-24
4,1.68750,707,2894,1111111111111111111111221212000000000221111112...,13257,2021-01-25T03:59:59-08:00,2021-01-24T04:00:00-08:00,2,703,0,...,51.0,59.0,0.0,74.0,32.0,7.0,82.0,50.0,97.0,2021-01-25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,1.65625,721,2859,1111112112111112222222222222222323222222212211...,12814,2021-04-14T03:59:59-07:00,2021-04-13T04:00:00-07:00,20,710,1,...,54.0,40.0,66.0,80.0,46.0,74.0,76.0,25.0,100.0,2021-04-15
80,1.40625,301,2400,1111111111111111111112111111111111111121121121...,5541,2021-04-15T03:59:59-07:00,2021-04-14T04:00:00-07:00,0,774,1,...,46.0,52.0,69.0,68.0,26.0,41.0,68.0,21.0,90.0,2021-04-16
81,1.37500,193,2324,1111111111111112111112221222222222233333222222...,3148,2021-04-16T03:59:59-07:00,2021-04-15T04:00:00-07:00,1,1062,3,...,35.0,51.0,57.0,86.0,40.0,23.0,1.0,17.0,96.0,2021-04-17
82,1.50000,550,2609,1111111111111111122111111111111111111221122211...,10674,2021-04-17T03:59:59-07:00,2021-04-16T04:00:00-07:00,2,451,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [71]:
#combining the readiness dataframe to the above combined dataframe, to make the full oura dataframe
oura_test_combined_final = oura_combined_1.join(oura_sleep, lsuffix='_act_1', rsuffix='_res_1')
oura_test_combined_final

,average_met,cal_active,cal_total,class_5min,daily_movement,day_end,day_start,high,inactive,inactivity_alerts,...,score_efficiency,score_latency,score_rem,score_total,summary_date_res_1,temperature_delta,temperature_deviation,timezone,total_res_1,temperature_trend_deviation
0,0.93750,23,1942,0000000000000000000000000000000000000000000000...,461,2021-01-21T03:59:59-08:00,2021-01-20T04:00:00-08:00,0,141,1,...,86.0,91.0,63.0,84.0,2021-01-21,-0.15,-0.15,-480.0,28830.0,NaN
1,1.34375,234,2311,2232332222222221222222322222333332322223311221...,4151,2021-01-22T03:59:59-08:00,2021-01-21T04:00:00-08:00,1,792,1,...,83.0,67.0,39.0,55.0,2021-01-22,-0.10,-0.10,-480.0,21420.0,0.01
2,1.34375,290,2350,1211222221111111122212211222222222222224434333...,5337,2021-01-23T03:59:59-08:00,2021-01-22T04:00:00-08:00,0,612,0,...,69.0,97.0,86.0,82.0,2021-01-23,-0.32,-0.32,-480.0,28410.0,0.01
3,1.15625,96,2048,2232222211122211111111111111111111112211111111...,1832,2021-01-24T03:59:59-08:00,2021-01-23T04:00:00-08:00,0,517,1,...,86.0,86.0,66.0,61.0,2021-01-24,-0.26,-0.26,-480.0,22890.0,0.02
4,1.68750,707,2894,1111111111111111111111221212000000000221111112...,13257,2021-01-25T03:59:59-08:00,2021-01-24T04:00:00-08:00,2,703,0,...,76.0,91.0,72.0,69.0,2021-01-27,0.16,0.16,-480.0,25020.0,0.28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,1.65625,721,2859,1111112112111112222222222222222323222222212211...,12814,2021-04-14T03:59:59-07:00,2021-04-13T04:00:00-07:00,20,710,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
80,1.40625,301,2400,1111111111111111111112111111111111111121121121...,5541,2021-04-15T03:59:59-07:00,2021-04-14T04:00:00-07:00,0,774,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
81,1.37500,193,2324,1111111111111112111112221222222222233333222222...,3148,2021-04-16T03:59:59-07:00,2021-04-15T04:00:00-07:00,1,1062,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
82,1.50000,550,2609,1111111111111111122111111111111111111221122211...,10674,2021-04-17T03:59:59-07:00,2021-04-16T04:00:00-07:00,2,451,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [72]:
#inspecting what columns to drop
oura_test_combined_final.columns

Index(['average_met', 'cal_active', 'cal_total', 'class_5min',
       'daily_movement', 'day_end', 'day_start', 'high', 'inactive',
       'inactivity_alerts', 'low', 'medium', 'met_1min', 'met_min_high',
       'met_min_inactive', 'met_min_low', 'met_min_medium', 'non_wear', 'rest',
       'rest_mode_state_act_1', 'score_act_1', 'score_meet_daily_targets',
       'score_move_every_hour', 'score_recovery_time', 'score_stay_active',
       'score_training_frequency', 'score_training_volume', 'steps',
       'summary_date_act', 'target_calories', 'target_km', 'target_miles',
       'timezone_act', 'to_target_km', 'to_target_miles', 'total_act_1',
       'bedtime_end_act_1', 'bedtime_start_act_1', 'breath_average_act_1',
       'duration_act_1', 'period_id_act_1', 'summary_date_res', 'timezone_res',
       'hr_average_act_1', 'hr_lowest_act_1', 'rmssd_act_1', 'period_id_res_1',
       'rest_mode_state_res_1', 'score_res_1', 'score_activity_balance',
       'score_hrv_balance', 'score_prev

In [73]:
#renaming columns with the correct unit of time
oura_test_combined_final.rename(columns={"inactive":"inactive_minutes"}, inplace=True)
oura_test_combined_final.rename(columns={"non_wear":"non_wear_minutes"}, inplace=True)
oura_test_combined_final.rename(columns={"rest":"rest_minutes"}, inplace=True)
oura_test_combined_final.rename(columns={"low":"low_minutes"}, inplace=True)
oura_test_combined_final.rename(columns={"medium":"medium_minutes"}, inplace=True)
oura_test_combined_final.rename(columns={"high":"high_minutes"}, inplace=True)
oura_test_combined_final.rename(columns={"awake":"awake_seconds"}, inplace=True)
oura_test_combined_final.rename(columns={"rem":"rem_seconds"}, inplace=True)
oura_test_combined_final.rename(columns={"light":"light_seconds"}, inplace=True)
oura_test_combined_final.rename(columns={"deep":"deep_seconds"}, inplace=True)
oura_test_combined_final.rename(columns={"temperature_delta":"temperature_delta_celsius"}, inplace=True)
oura_test_combined_final.rename(columns={"cal_total":"cal_total_kcal"}, inplace=True)

In [74]:
#checking for numerical anomalies

oura_test_combined_final["inactive_minutes"].describe()

count      84.000000
mean      623.785714
std       182.630846
min       141.000000
25%       516.500000
50%       648.500000
75%       745.250000
max      1062.000000
Name: inactive_minutes, dtype: float64

In [75]:
#checking for numerical anomalies

oura_test_combined_final["medium_minutes"].describe()

count     84.000000
mean      28.119048
std       24.372315
min        2.000000
25%       10.000000
50%       22.000000
75%       38.000000
max      119.000000
Name: medium_minutes, dtype: float64

In [76]:
# checking for numerical anomalies

oura_test_combined_final["daily_movement"].describe()

count       84.000000
mean      6216.178571
std       4326.205648
min        461.000000
25%       3137.750000
50%       5568.500000
75%       7763.500000
max      22091.000000
Name: daily_movement, dtype: float64

In [77]:
#checking for numerical anomalies

oura_test_combined_final["score_resting_hr"].describe()

count     82.000000
mean      80.256098
std       23.369185
min        1.000000
25%       70.000000
50%       88.500000
75%       99.000000
max      100.000000
Name: score_resting_hr, dtype: float64

In [78]:
#checking for numerical anomalies in the

oura_test_combined_final[oura_test_combined_final["daily_movement"] > 20000]["daily_movement"]

71    22091
Name: daily_movement, dtype: int64

As a note, the summary_date column is calculated as activity period between 4am - 3:59am user's local time. With day_start == 4am local time, and day_end == 3:59am local time

After going through different numerical columns we concluded that most data is correct, including any extremes found. Data that we are unsure of (such as the heart rate) will be left alone for the time being as we are unsure of how it is calculated and if we will be using it. If we decide to use it we will figure out how the values were calculated, and then remove it if necessary.

Going through the API documentation, it became apparent that the value '0' was often put in as a substitute for null. I will be changing these values to null.

In [79]:
# creating a function that changes the value 0 to null in places where 0 represents no available data

def replace_zero(column):
    for index in range(len(column)):
        if column[index] == 0:
            column[index] = np.nan

In [80]:
#replacing all zeros that are representative of null to null
replace_zero(oura_test_combined_final["score_recovery_time"])
replace_zero(oura_test_combined_final["score_training_volume"])
replace_zero(oura_test_combined_final["score_training_frequency"])
replace_zero(oura_test_combined_final["score_meet_daily_targets"])
replace_zero(oura_test_combined_final["score_move_every_hour"])
replace_zero(oura_test_combined_final["score_stay_active"])
replace_zero(oura_test_combined_final["score"])
replace_zero(oura_test_combined_final["score_total"])
replace_zero(oura_test_combined_final["score_rem"])
replace_zero(oura_test_combined_final["score_efficiency"])
replace_zero(oura_test_combined_final["score_deep"])
replace_zero(oura_test_combined_final["score_latency"])
replace_zero(oura_test_combined_final["score_disturbances"])
replace_zero(oura_test_combined_final["score_alignment"])

In [81]:
oura_test_combined_final["score_alignment"].value_counts()

100.0    46
94.0      2
88.0      2
85.0      2
32.0      1
9.0       1
69.0      1
1.0       1
41.0      1
31.0      1
45.0      1
65.0      1
84.0      1
78.0      1
70.0      1
61.0      1
89.0      1
91.0      1
71.0      1
87.0      1
37.0      1
86.0      1
98.0      1
72.0      1
96.0      1
68.0      1
Name: score_alignment, dtype: int64

In [82]:
oura_test_combined_final["summary_date_act"].value_counts()

2021-04-07    1
2021-02-13    1
2021-02-10    1
2021-03-02    1
2021-04-03    1
             ..
2021-02-04    1
2021-02-11    1
2021-03-15    1
2021-02-25    1
2021-02-24    1
Name: summary_date_act, Length: 84, dtype: int64

In [83]:
twitter_tweets = pd.read_json("data/ore_tweets_content.json")
twitter_tweets

,tweet
0,"{'retweeted': False, 'source': '<a href=""http:..."
1,"{'retweeted': False, 'source': '<a href=""http:..."
2,"{'retweeted': False, 'source': '<a href=""http:..."
3,"{'retweeted': False, 'source': '<a href=""http:..."
4,"{'retweeted': False, 'source': '<a href=""http:..."
...,...
2699,"{'retweeted': False, 'source': '<a href=""http:..."
2700,"{'retweeted': False, 'source': '<a href=""http:..."
2701,"{'retweeted': False, 'source': '<a href=""http:..."
2702,"{'retweeted': False, 'source': '<a href=""http:..."


In [84]:
#splitting the above dataframe

twitter_tweets_split = twitter_tweets['tweet'].apply(pd.Series)
twitter_tweets_split


,retweeted,source,entities,display_text_range,favorite_count,in_reply_to_status_id_str,id_str,in_reply_to_user_id,truncated,retweet_count,id,in_reply_to_status_id,created_at,favorited,full_text,lang,in_reply_to_screen_name,in_reply_to_user_id_str,possibly_sensitive,extended_entities
0,False,"<a href=""http://twitter.com/download/iphone"" r...","{'hashtags': [], 'symbols': [], 'user_mentions...","[0, 50]",0,1031515975928213505,1031526032153735168,29176998,False,0,1031526032153735168,1031515975928213505,Mon Aug 20 12:59:05 +0000 2018,False,@silentworks @jakuuire @pharingee Nice! I’m ga...,en,silentworks,29176998,NaN,NaN
1,False,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...","{'hashtags': [], 'symbols': [], 'user_mentions...","[0, 16]",0,1031500040408387584,1031503428672008193,29176998,False,0,1031503428672008193,1031500040408387584,Mon Aug 20 11:29:16 +0000 2018,False,@silentworks 💀💀💀,und,silentworks,29176998,NaN,NaN
2,False,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...","{'hashtags': [], 'symbols': [], 'user_mentions...","[0, 140]",0,NaN,1031263237025812485,NaN,False,0,1031263237025812485,NaN,Sun Aug 19 19:34:50 +0000 2018,False,RT @IbukunOg: Benjamin @DadaBen_ ! I’m so prou...,en,NaN,NaN,NaN,NaN
3,False,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...","{'hashtags': [], 'symbols': [], 'user_mentions...","[0, 85]",0,NaN,1031262871240486912,NaN,False,0,1031262871240486912,NaN,Sun Aug 19 19:33:23 +0000 2018,False,RT @Mz_Chi: Here is a link to the video if you...,en,NaN,NaN,False,NaN
4,False,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...","{'hashtags': [], 'symbols': [], 'user_mentions...","[0, 121]",0,NaN,1031262859899154432,NaN,False,0,1031262859899154432,NaN,Sun Aug 19 19:33:20 +0000 2018,False,RT @Mz_Chi: I'm going to make a YouTube video ...,en,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2699,False,"<a href=""http://twitter.com/download/iphone"" r...","{'hashtags': [], 'symbols': [], 'user_mentions...","[0, 140]",0,NaN,1236903007364632578,NaN,False,0,1236903007364632578,NaN,Mon Mar 09 06:33:52 +0000 2020,False,RT @DavisVilums: Exactly 5 years ago I started...,en,NaN,NaN,NaN,NaN
2700,False,"<a href=""http://twitter.com/download/iphone"" r...","{'hashtags': [], 'symbols': [], 'user_mentions...","[0, 97]",0,NaN,1236902584012509184,NaN,False,0,1236902584012509184,NaN,Mon Mar 09 06:32:11 +0000 2020,False,RT @paulbatum: Come work with me on Azure Func...,en,NaN,NaN,False,NaN
2701,False,"<a href=""http://twitter.com/download/iphone"" r...","{'hashtags': [{'text': 'dataviz', 'indices': [...","[0, 140]",0,NaN,1235819868873605120,NaN,False,0,1235819868873605120,NaN,Fri Mar 06 06:49:51 +0000 2020,False,RT @sxywu: ICYMI MY LATEST🎊is a physical #data...,en,NaN,NaN,NaN,NaN
2702,False,"<a href=""http://twitter.com/download/iphone"" r...","{'hashtags': [], 'symbols': [], 'user_mentions...","[0, 59]",2,1235524406778032128,1235568746711756801,29176998,False,1,1235568746711756801,1235524406778032128,Thu Mar 05 14:11:59 +0000 2020,False,@silentworks https://t.co/mBQ2Zx0Rqt\n\nYou kn...,en,silentworks,29176998,False,NaN


In [85]:
#splitting different user's mentioned into the same column
def get_user_mentions(entities):
    if entities is None:
        return []
    
    user_mentions_list = entities["user_mentions"]

    return [ single_mention["screen_name"] for single_mention in user_mentions_list]

twitter_tweets_split["users_mentioned"] = twitter_tweets_split["entities"].apply(get_user_mentions)

twitter_tweets_split["users_mentioned"]

0       [silentworks, jakuuire, pharingee]
1                            [silentworks]
2                     [IbukunOg, DadaBen_]
3                                 [Mz_Chi]
4                                 [Mz_Chi]
                       ...                
2699                         [DavisVilums]
2700                           [paulbatum]
2701                               [sxywu]
2702                         [silentworks]
2703                         [emeka_boris]
Name: users_mentioned, Length: 2704, dtype: object

In [86]:
#final tweets dataframe
twitter_tweets_final = twitter_tweets_split
twitter_tweets_final.columns

Index(['retweeted', 'source', 'entities', 'display_text_range',
       'favorite_count', 'in_reply_to_status_id_str', 'id_str',
       'in_reply_to_user_id', 'truncated', 'retweet_count', 'id',
       'in_reply_to_status_id', 'created_at', 'favorited', 'full_text', 'lang',
       'in_reply_to_screen_name', 'in_reply_to_user_id_str',
       'possibly_sensitive', 'extended_entities', 'users_mentioned'],
      dtype='object')

Here I will be performin sentiment analysis on the tweets. To do so I will first need to clean the contents of the tweets - specifically get rid of any text that says "RT" or  user names.

In [87]:
#Creating new dataframe and new features
twitter_tweets_sentiment = pd.DataFrame(twitter_tweets_final)
twitter_tweets_sentiment["clean_text"] = twitter_tweets_sentiment["full_text"]
twitter_tweets_sentiment


,retweeted,source,entities,display_text_range,favorite_count,in_reply_to_status_id_str,id_str,in_reply_to_user_id,truncated,retweet_count,...,created_at,favorited,full_text,lang,in_reply_to_screen_name,in_reply_to_user_id_str,possibly_sensitive,extended_entities,users_mentioned,clean_text
0,False,"<a href=""http://twitter.com/download/iphone"" r...","{'hashtags': [], 'symbols': [], 'user_mentions...","[0, 50]",0,1031515975928213505,1031526032153735168,29176998,False,0,...,Mon Aug 20 12:59:05 +0000 2018,False,@silentworks @jakuuire @pharingee Nice! I’m ga...,en,silentworks,29176998,NaN,NaN,"[silentworks, jakuuire, pharingee]",@silentworks @jakuuire @pharingee Nice! I’m ga...
1,False,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...","{'hashtags': [], 'symbols': [], 'user_mentions...","[0, 16]",0,1031500040408387584,1031503428672008193,29176998,False,0,...,Mon Aug 20 11:29:16 +0000 2018,False,@silentworks 💀💀💀,und,silentworks,29176998,NaN,NaN,[silentworks],@silentworks 💀💀💀
2,False,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...","{'hashtags': [], 'symbols': [], 'user_mentions...","[0, 140]",0,NaN,1031263237025812485,NaN,False,0,...,Sun Aug 19 19:34:50 +0000 2018,False,RT @IbukunOg: Benjamin @DadaBen_ ! I’m so prou...,en,NaN,NaN,NaN,NaN,"[IbukunOg, DadaBen_]",RT @IbukunOg: Benjamin @DadaBen_ ! I’m so prou...
3,False,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...","{'hashtags': [], 'symbols': [], 'user_mentions...","[0, 85]",0,NaN,1031262871240486912,NaN,False,0,...,Sun Aug 19 19:33:23 +0000 2018,False,RT @Mz_Chi: Here is a link to the video if you...,en,NaN,NaN,False,NaN,[Mz_Chi],RT @Mz_Chi: Here is a link to the video if you...
4,False,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...","{'hashtags': [], 'symbols': [], 'user_mentions...","[0, 121]",0,NaN,1031262859899154432,NaN,False,0,...,Sun Aug 19 19:33:20 +0000 2018,False,RT @Mz_Chi: I'm going to make a YouTube video ...,en,NaN,NaN,NaN,NaN,[Mz_Chi],RT @Mz_Chi: I'm going to make a YouTube video ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2699,False,"<a href=""http://twitter.com/download/iphone"" r...","{'hashtags': [], 'symbols': [], 'user_mentions...","[0, 140]",0,NaN,1236903007364632578,NaN,False,0,...,Mon Mar 09 06:33:52 +0000 2020,False,RT @DavisVilums: Exactly 5 years ago I started...,en,NaN,NaN,NaN,NaN,[DavisVilums],RT @DavisVilums: Exactly 5 years ago I started...
2700,False,"<a href=""http://twitter.com/download/iphone"" r...","{'hashtags': [], 'symbols': [], 'user_mentions...","[0, 97]",0,NaN,1236902584012509184,NaN,False,0,...,Mon Mar 09 06:32:11 +0000 2020,False,RT @paulbatum: Come work with me on Azure Func...,en,NaN,NaN,False,NaN,[paulbatum],RT @paulbatum: Come work with me on Azure Func...
2701,False,"<a href=""http://twitter.com/download/iphone"" r...","{'hashtags': [{'text': 'dataviz', 'indices': [...","[0, 140]",0,NaN,1235819868873605120,NaN,False,0,...,Fri Mar 06 06:49:51 +0000 2020,False,RT @sxywu: ICYMI MY LATEST🎊is a physical #data...,en,NaN,NaN,NaN,NaN,[sxywu],RT @sxywu: ICYMI MY LATEST🎊is a physical #data...
2702,False,"<a href=""http://twitter.com/download/iphone"" r...","{'hashtags': [], 'symbols': [], 'user_mentions...","[0, 59]",2,1235524406778032128,1235568746711756801,29176998,False,1,...,Thu Mar 05 14:11:59 +0000 2020,False,@silentworks https://t.co/mBQ2Zx0Rqt\n\nYou kn...,en,silentworks,29176998,False,NaN,[silentworks],@silentworks https://t.co/mBQ2Zx0Rqt\n\nYou kn...


In [88]:
#Identifying tweets that are retweets
for index in range(len(twitter_tweets_sentiment)):
    if bool("RT" in twitter_tweets_sentiment["clean_text"][index]) == True:
        twitter_tweets_sentiment["retweeted"][index] = True
    else:
        twitter_tweets_sentiment["retweeted"][index] = False
        

<ipython-input-88-428b0ad6d3ca>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  twitter_tweets_sentiment["retweeted"][index] = False
<ipython-input-88-428b0ad6d3ca>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  twitter_tweets_sentiment["retweeted"][index] = True


In [89]:
twitter_tweets_sentiment["retweeted"].value_counts()

True     1841
False     863
Name: retweeted, dtype: int64

In [90]:
#Removing RT, Punctuation etc
#create seperate column for RT

twitter_tweets_sentiment["clean_text"]= twitter_tweets_sentiment["clean_text"].replace(to_replace ='[RT]', value = '', regex = True)
twitter_tweets_sentiment["clean_text"]= twitter_tweets_sentiment["clean_text"].replace(to_replace ='@[\w:]+', value = '', regex = True)
twitter_tweets_sentiment["clean_text"] = twitter_tweets_sentiment.clean_text.str.lower()
twitter_tweets_sentiment["clean_text"][0:20]

0                                      nice! i’m game 💯
1                                                   💀💀💀
2       benjamin  ! i’m so proud of you! i see this ...
3       here is a link to the video if you be intere...
4       i'm going to make a youube video showing how...
5            from dusk till dawn https://t.co/bziziam9w
6       "maybe you don't understand"\nẹgbàmí, what w...
7       here is something that i discovered quite ea...
8       woohoo! my first linkedin article sharing my...
9       update!: #forlooppod is now available on iun...
10      at  2.0., one thing you are certain of getti...
11       oh: “be like a compiler, and ignore comments.”
12      important notice to developers of software t...
13      his would be good for    😂 and we can put  o...
14       hello ess,\ni am working on my master's deg...
15      when you remember that one edge case that yo...
16      on your first day at the new job, squash eve...
17      do you like statistics? do you dislike s

In [91]:
#remove emojis
def remove_emoji(text):
    return emoji.get_emoji_regexp().sub(u'', text)

twitter_tweets_sentiment["clean_text"] = twitter_tweets_sentiment["clean_text"].apply(remove_emoji)

In [92]:
#remove links
links = "((https?):((//)|(\\\\))+([\w\d:#@%/;$()~_?\+-=\\\.&](#!)?)*)"
twitter_tweets_sentiment["clean_text"]= twitter_tweets_sentiment["clean_text"].replace(to_replace =links, value = '', regex = True)
twitter_tweets_sentiment["clean_text"][156]

'  his is part of a full day of ai programming worldwide! join us on wednesday at '

In [93]:
#remove punctuation
twitter_tweets_sentiment["clean_text"]= twitter_tweets_sentiment["clean_text"].replace(to_replace =r'[^\w\s]', value = '', regex = True)




In [94]:
#remove double white spaces
twitter_tweets_sentiment["clean_text"]= twitter_tweets_sentiment["clean_text"].replace(to_replace =r'[\s]{2,}', value = ' ', regex = True)


In [95]:
#remove white space from the start of sentences

twitter_tweets_sentiment["clean_text"] = twitter_tweets_sentiment["clean_text"].apply(lambda x: x.strip())
twitter_tweets_sentiment["clean_text"]

0                                            nice im game
1                                                        
2       benjamin im so proud of you i see this guy tak...
3        here is a link to the video if you be interested
4       im going to make a youube video showing how yo...
                              ...                        
2699    exactly 5 years ago i started my journey to vi...
2700    come work with me on azure functions for more ...
2701    icymi my laesis a physical dataviz of women in...
2702                                  you know what to do
2703                                          yes you can
Name: clean_text, Length: 2704, dtype: object

In [96]:
twitter_tweets_sentiment["full_text"][13]

'RT @Babajiide: This Would be good for @codebeast @segunfamisa @Soloxpress 😂 and we can put @unicodeveloper on watch list as he as a babe no…'

In [97]:
#Calculating sentiment using TextBlob. Polarity in this case is meant to show how positive or negative the tweet was, with super positive being 1 and negative being -1.

twitter_tweets_sentiment[['polarity', 'subjectivity']] = twitter_tweets_sentiment["clean_text"].apply(lambda text: pd.Series(TextBlob(text).sentiment))
twitter_tweets_sentiment[0:10]

,retweeted,source,entities,display_text_range,favorite_count,in_reply_to_status_id_str,id_str,in_reply_to_user_id,truncated,retweet_count,...,full_text,lang,in_reply_to_screen_name,in_reply_to_user_id_str,possibly_sensitive,extended_entities,users_mentioned,clean_text,polarity,subjectivity
0,False,"<a href=""http://twitter.com/download/iphone"" r...","{'hashtags': [], 'symbols': [], 'user_mentions...","[0, 50]",0,1031515975928213505,1031526032153735168,29176998,False,0,...,@silentworks @jakuuire @pharingee Nice! I’m ga...,en,silentworks,29176998,NaN,NaN,"[silentworks, jakuuire, pharingee]",nice im game,0.100000,0.700000
1,False,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...","{'hashtags': [], 'symbols': [], 'user_mentions...","[0, 16]",0,1031500040408387584,1031503428672008193,29176998,False,0,...,@silentworks 💀💀💀,und,silentworks,29176998,NaN,NaN,[silentworks],,0.000000,0.000000
2,True,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...","{'hashtags': [], 'symbols': [], 'user_mentions...","[0, 140]",0,NaN,1031263237025812485,NaN,False,0,...,RT @IbukunOg: Benjamin @DadaBen_ ! I’m so prou...,en,NaN,NaN,NaN,NaN,"[IbukunOg, DadaBen_]",benjamin im so proud of you i see this guy tak...,0.245455,0.684848
3,True,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...","{'hashtags': [], 'symbols': [], 'user_mentions...","[0, 85]",0,NaN,1031262871240486912,NaN,False,0,...,RT @Mz_Chi: Here is a link to the video if you...,en,NaN,NaN,False,NaN,[Mz_Chi],here is a link to the video if you be interested,0.250000,0.500000
4,True,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...","{'hashtags': [], 'symbols': [], 'user_mentions...","[0, 121]",0,NaN,1031262859899154432,NaN,False,0,...,RT @Mz_Chi: I'm going to make a YouTube video ...,en,NaN,NaN,NaN,NaN,[Mz_Chi],im going to make a youube video showing how yo...,0.000000,0.000000
5,True,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...","{'user_mentions': [{'name': 'The Chi', 'screen...","[0, 55]",0,NaN,1031262697109835777,NaN,False,0,...,RT @Mz_Chi: From dusk till dawn https://t.co/B...,en,NaN,NaN,False,{'media': [{'expanded_url': 'https://twitter.c...,[Mz_Chi],from dusk till dawn,0.000000,0.000000
6,True,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...","{'hashtags': [], 'symbols': [], 'user_mentions...","[0, 115]",0,NaN,1031205963964067842,NaN,False,0,...,"RT @allenakinkunle: ""Maybe you don't understan...",ca,NaN,NaN,False,NaN,[allenakinkunle],maybe you dont understand\nẹgbàmí what was he ...,0.000000,0.000000
7,True,"<a href=""http://twitter.com/download/iphone"" r...","{'hashtags': [], 'symbols': [], 'user_mentions...","[0, 139]",0,NaN,1031147154709069824,NaN,False,0,...,RT @asemota: There is something that I discove...,en,NaN,NaN,NaN,NaN,[asemota],here is something that i discovered quite earl...,0.152083,0.360417
8,True,"<a href=""http://twitter.com/download/iphone"" r...","{'hashtags': [{'text': 'WISPDEFCONSQUAD', 'ind...","[0, 120]",0,NaN,1031074729610895360,NaN,False,0,...,RT @Ebunsky: Woohoo! My first LinkedIn article...,en,NaN,NaN,False,NaN,[Ebunsky],woohoo my first linkedin article sharing my de...,0.250000,0.333333
9,True,"<a href=""http://twitter.com/download/iphone"" r...","{'hashtags': [{'text': 'forLoopPod', 'indices'...","[0, 140]",0,NaN,1030796350580031489,NaN,False,0,...,RT @forLoopAfrica: Update!: #forLoopPod is now...,en,NaN,NaN,NaN,NaN,[forloopAfrica],update forlooppod is now available on iunes no...,0.300000,0.400000


Twitter has multiple timestamps per day (as you can tweet more than once)
GCalender multiple timestamps per day (more than one meetings)
Oura data one per day

In [98]:
#Calculating Negative, Positive, Neutral and Compound values using Vader
import nltk
nltk.download('vader_lexicon')

for index, row in twitter_tweets_sentiment["clean_text"].iteritems():
    score = SentimentIntensityAnalyzer().polarity_scores(row)
    neg = score["neg"]
    neu = score["neu"]
    pos = score["pos"]
    comp = score["compound"]
    if score["neg"] > score["pos"]:
        twitter_tweets_sentiment.loc[index, "sentiment"] = 'negative'
    elif score["pos"] > score["neg"]:
        twitter_tweets_sentiment.loc[index, "sentiment"] = 'positive'
    else:
        twitter_tweets_sentiment.loc[index, "sentiment"] = 'neutral'
    twitter_tweets_sentiment.loc[index, "compound"] = score["compound"]

twitter_tweets_sentiment.head(10)

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\oreog\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


,retweeted,source,entities,display_text_range,favorite_count,in_reply_to_status_id_str,id_str,in_reply_to_user_id,truncated,retweet_count,...,in_reply_to_screen_name,in_reply_to_user_id_str,possibly_sensitive,extended_entities,users_mentioned,clean_text,polarity,subjectivity,sentiment,compound
0,False,"<a href=""http://twitter.com/download/iphone"" r...","{'hashtags': [], 'symbols': [], 'user_mentions...","[0, 50]",0,1031515975928213505,1031526032153735168,29176998,False,0,...,silentworks,29176998,NaN,NaN,"[silentworks, jakuuire, pharingee]",nice im game,0.100000,0.700000,positive,0.4215
1,False,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...","{'hashtags': [], 'symbols': [], 'user_mentions...","[0, 16]",0,1031500040408387584,1031503428672008193,29176998,False,0,...,silentworks,29176998,NaN,NaN,[silentworks],,0.000000,0.000000,neutral,0.0000
2,True,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...","{'hashtags': [], 'symbols': [], 'user_mentions...","[0, 140]",0,NaN,1031263237025812485,NaN,False,0,...,NaN,NaN,NaN,NaN,"[IbukunOg, DadaBen_]",benjamin im so proud of you i see this guy tak...,0.245455,0.684848,positive,0.6113
3,True,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...","{'hashtags': [], 'symbols': [], 'user_mentions...","[0, 85]",0,NaN,1031262871240486912,NaN,False,0,...,NaN,NaN,False,NaN,[Mz_Chi],here is a link to the video if you be interested,0.250000,0.500000,positive,0.4019
4,True,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...","{'hashtags': [], 'symbols': [], 'user_mentions...","[0, 121]",0,NaN,1031262859899154432,NaN,False,0,...,NaN,NaN,NaN,NaN,[Mz_Chi],im going to make a youube video showing how yo...,0.000000,0.000000,negative,-0.1280
5,True,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...","{'user_mentions': [{'name': 'The Chi', 'screen...","[0, 55]",0,NaN,1031262697109835777,NaN,False,0,...,NaN,NaN,False,{'media': [{'expanded_url': 'https://twitter.c...,[Mz_Chi],from dusk till dawn,0.000000,0.000000,neutral,0.0000
6,True,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...","{'hashtags': [], 'symbols': [], 'user_mentions...","[0, 115]",0,NaN,1031205963964067842,NaN,False,0,...,NaN,NaN,False,NaN,[allenakinkunle],maybe you dont understand\nẹgbàmí what was he ...,0.000000,0.000000,neutral,0.0000
7,True,"<a href=""http://twitter.com/download/iphone"" r...","{'hashtags': [], 'symbols': [], 'user_mentions...","[0, 139]",0,NaN,1031147154709069824,NaN,False,0,...,NaN,NaN,NaN,NaN,[asemota],here is something that i discovered quite earl...,0.152083,0.360417,positive,0.8176
8,True,"<a href=""http://twitter.com/download/iphone"" r...","{'hashtags': [{'text': 'WISPDEFCONSQUAD', 'ind...","[0, 120]",0,NaN,1031074729610895360,NaN,False,0,...,NaN,NaN,False,NaN,[Ebunsky],woohoo my first linkedin article sharing my de...,0.250000,0.333333,positive,0.7269
9,True,"<a href=""http://twitter.com/download/iphone"" r...","{'hashtags': [{'text': 'forLoopPod', 'indices'...","[0, 140]",0,NaN,1030796350580031489,NaN,False,0,...,NaN,NaN,NaN,NaN,[forloopAfrica],update forlooppod is now available on iunes no...,0.300000,0.400000,positive,0.2263


In [99]:
#drop empty rows



In [100]:
#example sentiment
print(twitter_tweets_sentiment["clean_text"][123])

#vader
print(twitter_tweets_sentiment["sentiment"][123])

#textblog
print(twitter_tweets_sentiment["polarity"][123])

last week wasnt too bad
positive
-0.3499999999999999


In [101]:
#final tweets dataframe

tweets_final_dataframe = twitter_tweets_sentiment

In [102]:
# get topics from tweets

# we can apply this to the cleaned tweets to see if there's something good there
tweets_final_dataframe["topics"] = tweets_final_dataframe["clean_text"].apply(lambda x: browsing_yake_extractor.extract_keywords(x))

In [103]:
tweets_final_dataframe["topics"][450]

[('walking freely today', 0.0042542192213185686),
 ('josiah was murdered', 0.015380821171891606),
 ('party in humboldt', 0.02570861714399338),
 ('white man', 0.02570861714399338),
 ('years ago', 0.02570861714399338)]

In [104]:
#Checking how many words contain short hand
shorthand = ["omg", "til", "tfwiw", "tgif", "icymi", "lol", "lool", "loool"]
count = 0

for word in shorthand:
    for text in twitter_tweets_sentiment["clean_text"]:
        if bool(word in text) == True:
            count = count + 1
            
count

70

Changing these words will need to be done when putting this into production

In [105]:
activity_watch_dataframe_final.columns

Index(['timestamp_utc', 'duration_seconds', 'app', 'title', 'keywords',
       'event_classification'],
      dtype='object')

## combine the rows together to form a single dataset 


activity_watch_summarized = summary_day, total_time_surfing, what_app_did_user_spend_x_percent_of_time_on (one column per percent) , events_usage (tuple: event_classification (encoded), sum_total_time)

In [106]:
oura_test_combined_final.columns

Index(['average_met', 'cal_active', 'cal_total_kcal', 'class_5min',
       'daily_movement', 'day_end', 'day_start', 'high_minutes',
       'inactive_minutes', 'inactivity_alerts', 'low_minutes',
       'medium_minutes', 'met_1min', 'met_min_high', 'met_min_inactive',
       'met_min_low', 'met_min_medium', 'non_wear_minutes', 'rest_minutes',
       'rest_mode_state_act_1', 'score_act_1', 'score_meet_daily_targets',
       'score_move_every_hour', 'score_recovery_time', 'score_stay_active',
       'score_training_frequency', 'score_training_volume', 'steps',
       'summary_date_act', 'target_calories', 'target_km', 'target_miles',
       'timezone_act', 'to_target_km', 'to_target_miles', 'total_act_1',
       'bedtime_end_act_1', 'bedtime_start_act_1', 'breath_average_act_1',
       'duration_act_1', 'period_id_act_1', 'summary_date_res', 'timezone_res',
       'hr_average_act_1', 'hr_lowest_act_1', 'rmssd_act_1', 'period_id_res_1',
       'rest_mode_state_res_1', 'score_res_1', 'scor

In [107]:
tweets_final_dataframe.columns

Index(['retweeted', 'source', 'entities', 'display_text_range',
       'favorite_count', 'in_reply_to_status_id_str', 'id_str',
       'in_reply_to_user_id', 'truncated', 'retweet_count', 'id',
       'in_reply_to_status_id', 'created_at', 'favorited', 'full_text', 'lang',
       'in_reply_to_screen_name', 'in_reply_to_user_id_str',
       'possibly_sensitive', 'extended_entities', 'users_mentioned',
       'clean_text', 'polarity', 'subjectivity', 'sentiment', 'compound',
       'topics'],
      dtype='object')

In [108]:
tweets_final_dataframe.iloc[1233]

retweeted                                                                False
source                       <a href="http://twitter.com/download/iphone" r...
entities                     {'hashtags': [], 'symbols': [], 'user_mentions...
display_text_range                                                     [0, 46]
favorite_count                                                               2
in_reply_to_status_id_str                                                  NaN
id_str                                                     1319278958156734470
in_reply_to_user_id                                                        NaN
truncated                                                                False
retweet_count                                                                2
id                                                         1319278958156734470
in_reply_to_status_id                                                      NaN
created_at                                      Thu 

In [109]:
g_calendar_final.columns

Index(['kind', 'etag', 'id', 'status', 'htmlLink', 'created', 'updated',
       'summary', 'creator', 'organizer', 'start', 'end', 'recurringEventId',
       'originalStartTime', 'visibility', 'iCalUID', 'sequence', 'reminders',
       'eventType', 'description', 'colorId', 'transparency', 'location',
       'attendees', 'guestsCanInviteOthers', 'privateCopy', 'source',
       'hangoutLink', 'conferenceData', 'extendedProperties',
       'guestsCanModify', 'guestsCanSeeOtherGuests', 'start_dateTime_utc',
       'timeZone_act', 'date_act', 'end_dateTime_utc', 'timeZone_res',
       'date_res'],
      dtype='object')

In [110]:
g_calendar_final.sample(10)

,kind,etag,id,status,htmlLink,created,updated,summary,creator,organizer,...,conferenceData,extendedProperties,guestsCanModify,guestsCanSeeOtherGuests,start_dateTime_utc,timeZone_act,date_act,end_dateTime_utc,timeZone_res,date_res
185,calendar#event,"""3228403934781000""",ac3gk0hrgh9im9tp203db3cd2s_20210415T020000Z,confirmed,https://www.google.com/calendar/event?eid=YWMz...,2021-01-20T12:30:44.000Z,2021-03-18T03:35:31.198Z,NeuroTech Hacking,"{'email': 'oreogundipe@gmail.com', 'self': True}","{'email': 'oreogundipe@gmail.com', 'self': True}",...,NaN,NaN,NaN,NaN,2021-04-14T19:00:00-07:00,America/Vancouver,NaN,2021-04-14T20:30:00-07:00,America/Vancouver,NaN
132,calendar#event,"""3222976715751000""",3pu3cntn8me9ckgl3me3pu7sb9_20210314T190000Z,confirmed,https://www.google.com/calendar/event?eid=M3B1...,2020-12-06T19:52:42.000Z,2021-03-14T03:25:53.253Z,Adaobi/Ore Weekend Sync,"{'email': 'oreogundipe@gmail.com', 'self': True}","{'email': 'oreogundipe@gmail.com', 'self': True}",...,"{'entryPoints': [{'entryPointType': 'video', '...",NaN,NaN,NaN,2021-03-14T12:00:00-07:00,America/Vancouver,NaN,2021-03-14T13:00:00-07:00,America/Vancouver,NaN
50,calendar#event,"""3225067660702000""",rganm03dr0t95v677lcj71ssoo,confirmed,https://www.google.com/calendar/event?eid=cmdh...,2021-02-05T14:03:50.000Z,2021-02-05T14:03:50.351Z,Imposter Syndrome in Software Engineering,"{'email': 'oreogundipe@gmail.com', 'self': True}","{'email': 'oreogundipe@gmail.com', 'self': True}",...,NaN,NaN,NaN,NaN,2021-02-11T10:00:00-08:00,NaN,NaN,2021-02-11T11:00:00-08:00,NaN,NaN
107,calendar#event,"""3227878782712000""",8mdt3atmbh8juob1m1iv4irr98,confirmed,https://www.google.com/calendar/event?eid=OG1k...,2021-02-14T19:40:52.000Z,2021-02-21T20:29:51.356Z,Umar Faruq Akinwunmi and Ọrẹ̀ Ògúndípẹ̀,"{'email': 'oreogundipe@gmail.com', 'self': True}","{'email': 'oreogundipe@gmail.com', 'self': True}",...,{'createRequest': {'requestId': '1b57bdf4-d275...,NaN,NaN,NaN,2021-02-21T12:30:00-08:00,NaN,NaN,2021-02-21T13:00:00-08:00,NaN,NaN
84,calendar#event,"""3225691056730000""",fev6bb43bdk2hn7ljsbn2at5bg_20210121T134500Z,confirmed,https://www.google.com/calendar/event?eid=ZmV2...,2021-01-20T12:30:44.000Z,2021-02-14T17:59:53.383Z,BrainDump Project Updates,"{'email': 'oreogundipe@gmail.com', 'self': True}","{'email': 'oreogundipe@gmail.com', 'self': True}",...,NaN,NaN,NaN,NaN,2021-01-21T05:45:00-08:00,America/Vancouver,NaN,2021-01-21T06:45:00-08:00,America/Vancouver,NaN
104,calendar#event,"""3227403442458000""",85pf1omdibd7uieghgtr7b1bs4,confirmed,https://www.google.com/calendar/event?eid=ODVw...,2021-02-19T02:28:41.000Z,2021-02-19T02:28:41.229Z,The cleanrooms where it happened - 10 years of...,"{'email': 'oreogundipe@gmail.com', 'self': True}","{'email': 'oreogundipe@gmail.com', 'self': True}",...,NaN,NaN,NaN,NaN,2021-02-25T17:00:00-08:00,NaN,NaN,2021-02-25T18:00:00-08:00,NaN,NaN
87,calendar#event,"""3225691056730000""",fev6bb43bdk2hn7ljsbn2at5bg_20210127T134500Z,confirmed,https://www.google.com/calendar/event?eid=ZmV2...,2021-01-20T12:30:44.000Z,2021-02-14T17:59:53.383Z,BrainDump Project Updates,"{'email': 'oreogundipe@gmail.com', 'self': True}","{'email': 'oreogundipe@gmail.com', 'self': True}",...,NaN,NaN,NaN,NaN,2021-01-27T05:45:00-08:00,America/Vancouver,NaN,2021-01-27T06:45:00-08:00,America/Vancouver,NaN
116,calendar#event,"""3229042140458000""",84ak0fgt67fh6ibhrqsci2qad0,confirmed,https://www.google.com/calendar/event?eid=ODRh...,2021-02-27T13:11:39.000Z,2021-02-28T14:04:30.229Z,Ada and Ọrẹ̀ Ògúndípẹ̀,"{'email': 'oreogundipe@gmail.com', 'self': True}","{'email': 'oreogundipe@gmail.com', 'self': True}",...,NaN,NaN,NaN,NaN,2021-02-28T06:30:00-08:00,NaN,NaN,2021-02-28T07:00:00-08:00,NaN,NaN
216,calendar#event,"""3235232815770000""",d7ocbgcqks135l97ah4ab9o63o,confirmed,https://www.google.com/calendar/event?eid=ZDdv...,2021-04-05T04:26:03.000Z,2021-04-05T09:53:27.885Z,Ore and Neo,{'email': 'neo@creativitykills.co'},{'email': 'neo@creativitykills.co'},...,{'createRequest': {'reque

In [111]:
# now we need to get an aggreate summarization on the existing dataste

activity_watch_dataframe_final[0:10]

,timestamp_utc,duration_seconds,app,title,keywords,event_classification
0,2021-02-27T15:07:29.345000+00:00,52.882,msedge.exe,ActivityWatch/activitywatch: The best free and...,"[(automated time tracker, 0.008757238666059378...",activitywatch
1,2021-02-27T15:08:23.428000+00:00,0.000,msedge.exe,https://github.com/ActivityWatch and 22 more p...,[],msedge.exe
2,2021-02-27T15:08:24.630000+00:00,23.357,msedge.exe,ActivityWatch and 22 more pages - Personal - M...,"[(activitywatch, 0.15831692877998726)]",activitywatch
3,2021-02-27T15:08:49.183000+00:00,19.718,msedge.exe,Sponsor @ActivityWatch on GitHub Sponsors and ...,"[(activitywatch on github, 0.02111487214632003...",github sponsors
4,2021-02-27T15:09:10.151000+00:00,4.846,SearchApp.exe,Search,NaN,SearchApp.exe
5,2021-02-27T15:09:16.200000+00:00,13.051,explorer.exe,NaN,NaN,explorer.exe
6,2021-02-27T15:09:30.308000+00:00,3.712,aw-qt.exe,aw-qt,NaN,aw-qt.exe
7,2021-02-27T15:09:35.229000+00:00,0.104,msedge.exe,Untitled and 23 more pages - Personal - Micros...,"[(untitled, 0.15831692877998726)]",untitled
8,2021-02-27T15:09:36.444000+00:00,0.089,msedge.exe,localhost:5600 and 23 more pages - Personal - ...,"[(localhost, 0.04491197687864554)]",localhost
9,2021-02-27T15:09:37.642000+00:00,22.687,msedge.exe,ActivityWatch and 23 more pages - Personal - M...,"[(activitywatch, 0.15831692877998726)]",activitywatch
